Estimate prediction intervals associated with a time series forecast. For an example using MapieTimeSeriesRegressor, see  
mapie documentation: https://mapie.readthedocs.io/en/latest/examples_regression/4-tutorials/plot_ts-tutorial.html

## Import libraries

In [ ]:
import warnings

skip_warnings = True
if skip_warnings:
    warnings.filterwarnings('ignore', category=FutureWarning)
    # Suppress specific warning related to the deprecated binary model format
    warnings.filterwarnings("ignore", message=".*deprecated binary model format.*")
    # Suppress specific warning from EconML
    warnings.filterwarnings("ignore", message="Co-variance matrix is underdetermined. Inference will be invalid!")
    # Suppress specific warning from MAPIE
    warnings.filterwarnings("ignore", message="WARNING: The predictions are ill-sorted.")

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from pathlib import Path
import random, copy
from functools import wraps
import datetime as dt
import json
import itertools
from packaging import version

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.cm as cm
from matplotlib import colormaps
import seaborn as sns
import plotly.express as px
from pandas.plotting import scatter_matrix

from datetime import datetime

from scipy.stats import yeojohnson, boxcox

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder, LabelBinarizer
from sklearn.preprocessing import QuantileTransformer, Normalizer, RobustScaler
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from category_encoders.woe import WOEEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.target_encoder import TargetEncoder
from category_encoders.glmm import GLMMEncoder
from category_encoders.wrapper import PolynomialWrapper
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import FunctionTransformer, PowerTransformer, KBinsDiscretizer
from imblearn import FunctionSampler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklego.preprocessing import ColumnSelector
from sklearn.pipeline import Pipeline, FeatureUnion
from imblearn.pipeline import Pipeline as PipelineImb
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, StratifiedKFold
from sklearn.decomposition import PCA

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils import shuffle, check_array

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn_quantile import RandomForestQuantileRegressor, ExtraTreesQuantileRegressor, KNeighborsQuantileRegressor
from mapie.regression import MapieRegressor
from mapie.regression import MapieQuantileRegressor
from mapie.subsample import Subsample
from mapie.metrics import coverage_width_based, regression_coverage_score, regression_mean_width_score, regression_mwi_score
from mapie.regression import MapieTimeSeriesRegressor
from mapie.subsample import BlockBootstrap
import MWIS_metric

from sklearn.inspection import permutation_importance
import shap

from econml.dml import LinearDML, CausalForestDML
from econml.orf import DMLOrthoForest
from sklearn.preprocessing import KBinsDiscretizer

## Import utils

In [ ]:
from common import copy_dataframe, format_time_column, merge_onehot_columns, add_interaction_columns, replace_inf_with_nan, drop_rows_with_nan
from common import change_column_names, strip_blanks_column_names, move_columns_to_end_of_dataframe, add_date_component_columns
from common import create_bins, get_shap_values_from_array, split_dataset_in_train_calibrate_test, get_regression_metrics
from common import set_sample_weights, set_baseline_model

from common import YeoJohnsonTargetTransformer, LogTargetTransformer, Log1pTransformerWithShift, RuleBasedModel

## Import path names

In [ ]:
from common import DATA_DIR, EXTERNAL_DIR, INTERIM_DIR, OUTPUT_DIR, FIGURES_DIR

## Settings

In [ ]:
from sklearn import set_config

sklearn_display = 'diagram'  # 'diagram' or 'text'
set_config(display=sklearn_display)

In [ ]:
import matplotlib

# Get initial Matplotlib figure settings
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
initial_settings = plt.rcParams.copy()  # to avoid shap overruling initial plot settings

In [ ]:
from model_settings import seed_value

np.random.seed(seed_value)

## Load dataset

In [ ]:
path_name = EXTERNAL_DIR

file_name = 'data_training_energy_euro_V2.csv'
full_path = path_name / file_name
print(f"LOAD: {str(full_path)}\n")
data_raw = pd.read_csv(full_path)

data_raw.info()

## Clean dataset

### Data settings

In [ ]:
from data_settings import time_col, categoricals_dict, colnames_dict

print(f"Name of time column:")
print(f"    '{time_col}'")
print(f"Onehot encoded columns:")
for col in categoricals_dict.keys():
    print(f"  - '{col}': {categoricals_dict[col]}")
print(f"New names for columns:")
for col in colnames_dict.keys():
    print(f"  - '{col}': '{colnames_dict[col]}'")

In [ ]:
from common import copy_dataframe, format_time_column, merge_onehot_columns, add_interaction_columns, replace_inf_with_nan, drop_rows_with_nan, reset_index_dataframe
from common import change_column_names, strip_blanks_column_names, move_columns_to_end_of_dataframe, add_date_component_columns, make_timeseries_equal_length
from common import add_sin_cos_feature_columns, add_lagged_columns, add_moving_average_columns, add_diff_target, drop_days_from_series
from model_settings import add_lagged_target_to_model, target_max_lags, target, add_lagged_swan_predictions_to_model, swan_max_lags
from model_settings import add_moving_average_target_to_model, add_moving_average_swan_to_model, ma_window
from model_settings import periods_for_bias_correction, columns_not_in_model
from data_settings import dates_to_drop


data_clean = (data_raw
    .pipe(copy_dataframe)
    .pipe(format_time_column, dates_col=time_col, format='%Y-%m-%d %H:%M:%S')
    .pipe(add_date_component_columns, dates_col=time_col)
    .pipe(merge_onehot_columns, columns_dict=categoricals_dict)
    .pipe(add_interaction_columns)
    .pipe(replace_inf_with_nan)
    .pipe(add_sin_cos_feature_columns, dates_col=time_col, period=periods_for_bias_correction)
    .pipe(change_column_names, columns_rename=colnames_dict)
    .pipe(strip_blanks_column_names)
    .pipe(add_diff_target, column_name=target, groupby_columns=['Location','Frequency'], sortby_column='Time')
    .pipe(add_lagged_columns, column_name=target, groupby_columns=['Location','Frequency'], sortby_column='Time', add_column=add_lagged_target_to_model, lags=target_max_lags, columns_to_drop=columns_not_in_model)
    .pipe(add_lagged_columns, column_name='SWAN prediction', groupby_columns=['Location','Frequency'], sortby_column='Time', add_column=add_lagged_swan_predictions_to_model, lags=swan_max_lags, columns_to_drop=columns_not_in_model)
    .pipe(add_moving_average_columns, column_name=target, groupby_columns=['Location','Frequency'], sortby_column='Time', add_column=add_moving_average_target_to_model, window=ma_window, shift_series=True, columns_to_drop=columns_not_in_model)
    .pipe(add_moving_average_columns, column_name='SWAN prediction', groupby_columns=['Location','Frequency'], sortby_column='Time', add_column=add_moving_average_swan_to_model, window=ma_window, shift_series=False, columns_to_drop=columns_not_in_model)
    .pipe(drop_days_from_series, dates_to_drop=dates_to_drop)
    .pipe(drop_rows_with_nan)
    # .pipe(move_columns_to_end_of_dataframe, column_names=['value','output'])
    # .pipe(make_timeseries_equal_length, dates_col='Time', locations_col='Location', frequencies_col='Frequency')
    .pipe(reset_index_dataframe, drop_original_index=True)
)

print('')
data_clean.info()

In [ ]:
data_clean.head()

## Build regression model

### Model settings

In [ ]:
from model_settings import train_size, calib_size, test_size, target, locations_to_use, frequencies_to_use, columns_not_in_model
from model_settings import split_method, binned_target_log, binned_target_nbins, leave_one_out_location, shuffle_rows
from model_settings import imputation_method, scaler_type, scaler_quantile_nquantiles, scaler_rowwise_norm
from model_settings import use_categorical_encoding, use_ordinal_encoding, use_numeric_transform, pca_transform_numeric, use_target_transform
from model_settings import ordinal_columns, continuous_to_ordinal_transform_nbins, categorical_columns, categorical_encode_method, numeric_transform_method
from model_settings import target_transform_method, target_yeojohnson_lambda, target_yeojohnson_nan_percentile
from model_settings import use_sample_weights, fit_weight_tails, fit_quantiles_tails
from model_settings import set_monotonic_constraints, features_monotone_increasing, features_monotone_decreasing, monotone_constraints_method_lgbm

print(f"train_size: {train_size}")
print(f"calib_size: {calib_size}")
print(f"test_size: {test_size}")
print(f"target: {target}")
print(f"locations_to_use: {locations_to_use}")
print(f"frequencies_to_use: {frequencies_to_use}")
print(f"split_method: {split_method}")
print(f"binned_target_log: {binned_target_log}")
print(f"binned_target_nbins: {binned_target_nbins}")
print(f"leave_one_out_location: {leave_one_out_location}")
print(f"shuffle_rows: {shuffle_rows}")
print(f"imputation_method: {imputation_method}")
print(f"scaler_type: {scaler_type}")
print(f"scaler_quantile_nquantiles: {scaler_quantile_nquantiles}")
print(f"scaler_rowwise_norm: {scaler_rowwise_norm}")
print(f"columns_not_in_model: {columns_not_in_model}")
print(f"use_categorical_encoding: {use_categorical_encoding}")
print(f"use_ordinal_encoding: {use_ordinal_encoding}")
print(f"use_numeric_transform: {use_numeric_transform}")
print(f"pca_transform_numeric: {pca_transform_numeric}")
print(f"use_target_transform: {use_target_transform}")
print(f"ordinal_columns: {ordinal_columns}")
print(f"continuous_to_ordinal_transform_nbins: {continuous_to_ordinal_transform_nbins}")
print(f"categorical_columns: {categorical_columns}")
print(f"categorical_encode_method: {categorical_encode_method}")
print(f"numeric_transform_method: {numeric_transform_method}")
print(f"target_transform_method: {target_transform_method}")
print(f"target_yeojohnson_lambda: {target_yeojohnson_lambda}")
print(f"target_yeojohnson_nan_percentile: {target_yeojohnson_nan_percentile}")
print(f"use_sample_weights: {use_sample_weights}")
print(f"fit_weight_tails: {fit_weight_tails}")
print(f"fit_quantiles_tails: {fit_quantiles_tails}")
print(f"set_monotonic_constraints: {set_monotonic_constraints}")
print(f"features_monotone_increasing: {features_monotone_increasing}")
print(f"features_monotone_decreasing: {features_monotone_decreasing}")
# print(f"monotone_constraints_method_lgbm: {monotone_constraints_method_lgbm}")

### Train(-calibrate)-test split

In [ ]:
data_clean.describe().transpose()

In [ ]:
from model_settings import train_size, calib_size, test_size, target, locations_to_use, frequencies_to_use, columns_not_in_model
from model_settings import split_method, split_dates_table, binned_target_log, binned_target_nbins, leave_one_out_location, shuffle_rows
from model_settings import imputation_method, scaler_type, scaler_quantile_nquantiles, scaler_rowwise_norm
from model_settings import use_categorical_encoding, use_ordinal_encoding, use_numeric_transform, pca_transform_numeric, use_target_transform
from model_settings import ordinal_columns, continuous_to_ordinal_transform_nbins, categorical_columns, categorical_encode_method, numeric_transform_method
from model_settings import target_transform_method, target_yeojohnson_lambda, target_yeojohnson_nan_percentile
from model_settings import use_sample_weights, fit_weight_tails, fit_quantiles_tails
from model_settings import set_monotonic_constraints, features_monotone_increasing, features_monotone_decreasing, monotone_constraints_method_lgbm

# override split method
# split_method = 'timeseries_split'
# split_method = 'leave_one_out'
split_method = 'dates_table'

# override split ratios
train_size = 0.6
calib_size = 0.2
test_size = 0.2

X = data_clean.copy().loc[(data_clean['Location'].isin(locations_to_use)) & (data_clean['Frequency'].isin(frequencies_to_use)), data_clean.drop(columns=target).columns]
y = data_clean.copy().loc[(data_clean['Location'].isin(locations_to_use)) & (data_clean['Frequency'].isin(frequencies_to_use)), target]

train_data, calib_data, test_data = split_dataset_in_train_calibrate_test(
    X, y, target, split_ratios=(train_size, calib_size, test_size), 
    method=split_method, seed_value=seed_value, loo_loc=leave_one_out_location,
    dates_table=split_dates_table, shuffle_rows=shuffle_rows,
)

X_train = train_data['X']
y_train = train_data['y']
X_calib = calib_data['X']
y_calib = calib_data['y']
X_test = test_data['X']
y_test = test_data['y']
# train_index = train_data['index']
# calib_index = calib_data['index']
# test_index = test_data['index']

# digits = 3
# print(f"Train size:       {np.round(X_train.shape[0] / (X_train.shape[0] + X_calib.shape[0] + X_test.shape[0]), digits)}")
# if calib_size > 0:
#     print(f"Calibration size: {np.round(X_calib.shape[0] / (X_train.shape[0] + X_calib.shape[0] + X_test.shape[0]), digits)}")
# print(f"Test size:        {np.round(X_test.shape[0] / (X_train.shape[0] + X_calib.shape[0] + X_test.shape[0]), digits)}")

digits = 3
print(f"Train size:       {X_train.shape[0]}")
if calib_size > 0:
    print(f"Calibration size: {X_calib.shape[0]}")
print(f"Test size:        {X_test.shape[0]}")

X_new, y_new = X_test.copy(), y_test.copy()

In [ ]:
print('Length of time series per location (train data)')
for loc_name in X['Location'].unique():
    print(f"  {loc_name}: {X_train.loc[X.Location==loc_name,'Time'].min()} - {X_train.loc[X.Location==loc_name,'Time'].max()}")
if calib_size > 0:
    print('\nLength of time series per location (calibration data)')
    for loc_name in X['Location'].unique():
        print(f"  {loc_name}: {X_calib.loc[X.Location==loc_name,'Time'].min()} - {X_calib.loc[X.Location==loc_name,'Time'].max()}")
print('\nLength of time series per location (test data)')
for loc_name in X['Location'].unique():
    print(f"  {loc_name}: {X_test.loc[X.Location==loc_name,'Time'].min()} - {X_test.loc[X.Location==loc_name,'Time'].max()}")

In [ ]:
print('Range of values per location (train data)')
for loc_name in X['Location'].unique():
    print(f"  {loc_name}: {np.round(y_train.loc[X_train.loc[X.Location==loc_name].index].min(),3)} - {np.round(y_train.loc[X_train.loc[X.Location==loc_name].index].max(),3)}")
if calib_size > 0:
    print('\nRange of values per location (calibration data)')
    for loc_name in X['Location'].unique():
        print(f"  {loc_name}: {np.round(y_calib.loc[X_calib.loc[X.Location==loc_name].index].min(),3)} - {np.round(y_calib.loc[X_calib.loc[X.Location==loc_name].index].max(),3)}")
print('\nRange of values per location (test data)')
for loc_name in X['Location'].unique():
    print(f"  {loc_name}: {np.round(y_test.loc[X_test.loc[X.Location==loc_name].index].min(),3)} - {np.round(y_test.loc[X_test.loc[X.Location==loc_name].index].max(),3)}")

### Model pipeline

In [ ]:
def regression_model_pipeline():
    global rgr_mdl

    # Define lists to hold preprocessing steps
    num_preprocessor, cat_preprocessor, discrete2ord_preprocessor, continuous2ord_preprocessor, drop_col_preprocessor = [], [], [], [], []

    # Feature type lists
    drop_var_names = columns_not_in_model
    discrete2ord_var_names = [col for col in Xtrain_columns if col in discrete_to_ordinal and col not in drop_var_names] if use_ordinal_encoding else []
    continuous2ord_var_names = [col for col in Xtrain_columns if col in continuous_to_ordinal and col not in drop_var_names] if use_ordinal_encoding else []
    cat_var_names = [col for col in Xtrain_columns if col in categorical_columns and col not in drop_var_names] if use_categorical_encoding else []
    num_var_names = [col for col in Xtrain_columns if col not in cat_var_names and col not in discrete2ord_var_names and col not in continuous2ord_var_names and col not in drop_var_names]

    # Get column indexes of feature types
    num_vars_idx = [Xtrain_columns.index(col) for col in num_var_names if col in Xtrain_columns]
    cat_vars_idx = [Xtrain_columns.index(col) for col in cat_var_names if col in Xtrain_columns]
    discrete2ord_vars_idx = [Xtrain_columns.index(col) for col in discrete2ord_var_names if col in Xtrain_columns]
    continuous2ord_vars_idx = [Xtrain_columns.index(col) for col in continuous2ord_var_names if col in Xtrain_columns]
    drop_vars_idx = [Xtrain_columns.index(col) for col in drop_var_names if col in Xtrain_columns]

    # Drop features
    drop_col_preprocessor.append(('column_dropper', FunctionTransformer(drop_columns_by_name, kw_args={'columns_to_drop': drop_var_names}, validate=False)))

    # Impute missing values
    if imputation_method in ['median', 'mean']:
        num_preprocessor.append(('imputer', SimpleImputer(strategy=imputation_method)))
        if use_ordinal_encoding:
            if discrete2ord_vars_idx:
                discrete2ord_preprocessor.append(('imputer', SimpleImputer(strategy=imputation_method)))
            if continuous2ord_vars_idx:
                continuous2ord_preprocessor.append(('imputer', SimpleImputer(strategy=imputation_method)))
    elif imputation_method == 'knn':
        num_preprocessor.append(('imputer', KNNImputer()))
        if use_ordinal_encoding:
            if discrete2ord_vars_idx:
                discrete2ord_preprocessor.append(('imputer', KNNImputer()))
            if continuous2ord_vars_idx:
                continuous2ord_preprocessor.append(('imputer', KNNImputer()))
    if imputation_method != 'drop':
        if use_categorical_encoding and cat_vars_idx:
            if categorical_encode_method in ['WOE', 'CatBoost']:
                cat_preprocessor.append(('astype_str', FunctionTransformer(safe_convert_to_str, validate=False)))  # WOE will transform all string columns
            cat_preprocessor.append(('imputer', SimpleImputer(strategy='most_frequent')))

    # Transform numeric feature values
    if use_numeric_transform:
        if numeric_transform_method == 'log':
            num_preprocessor.append(('transformer', Log1pTransformerWithShift()))
        elif numeric_transform_method == 'signed_log':
            num_preprocessor.append(('transformer', FunctionTransformer(func=signed_log1p_transform, inverse_func=inv_signed_log1p_transform, validate=False)))
        elif numeric_transform_method == 'yeojohnson':
            num_preprocessor.append(('transformer', PowerTransformer(method='yeo-johnson')))
        else:
            raise ValueError(f"Unsupported numeric_transform_method for numeric features: {numeric_transform_method}")

    # Onehot encode categorical features
    if use_categorical_encoding and cat_vars_idx:
        if categorical_encode_method == 'OHE':
            if version.parse(sklearn.__version__) >= version.parse('1.2'):
                cat_preprocessor.append(('onehot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False)))
            else:
                cat_preprocessor.append(('onehot_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False)))
        elif categorical_encode_method == 'target':
            cat_preprocessor.append(('target_encoder', TargetEncoder(verbose=0, handle_unknown='value', handle_missing='value', return_df=False, drop_invariant=False)))
        elif categorical_encode_method == 'GLMM':
            cat_preprocessor.append(('glmm_encoder', GLMMEncoder(verbose=0, handle_unknown='value', handle_missing='value', return_df=False, drop_invariant=False)))
        elif categorical_encode_method == 'WOE':
            cat_preprocessor.append(('woe_encoder', WOEEncoder(verbose=0, handle_unknown='value', handle_missing='value', return_df=False, drop_invariant=False)))
        elif categorical_encode_method == 'CatBoost':
            cat_preprocessor.append(('catboost_encoder', CatBoostEncoder(verbose=0, handle_unknown='value', handle_missing='value', return_df=False, drop_invariant=False)))
        else:
            raise ValueError(f"Unsupported categorical_encode_method for features: {categorical_encode_method}")

    # Ordinal encode selected numeric features
    if use_ordinal_encoding:
        if discrete2ord_vars_idx:
            discrete2ord_preprocessor.append(('ordinal_encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)))
        if continuous2ord_vars_idx:
            nbins_discretizer = [continuous_to_ordinal_transform_nbins.get(name, 10) for name in continuous2ord_var_names]
            continuous2ord_preprocessor.append(('ordinal_discretizer', KBinsDiscretizer(n_bins=nbins_discretizer, encode='ordinal', strategy='uniform')))

    # Scale numeric (and ordinal) feature values
    scl_type = models[mdl_type].get('scaler', None)
    add_pca_to_pipe = models[mdl_type].get('pca_transform', None)
    scaler = None
    if scl_type == 'standard':
        scaler = StandardScaler()
    elif scl_type == 'minmax' and not pca_transform_numeric:
        scaler = MinMaxScaler()
    elif scl_type == 'quantile' and not pca_transform_numeric:
        scaler = QuantileTransformer(n_quantiles=scaler_quantile_nquantiles)
    elif scl_type == 'robust' and not pca_transform_numeric:
        scaler = RobustScaler()
    elif scl_type == 'rowwise' and not pca_transform_numeric:
        scaler = Normalizer(norm=scaler_rowwise_norm)
    if scaler:
        num_preprocessor.append(('scaler', scaler))
        if not use_categorical_encoding and cat_vars_idx:
            cat_preprocessor.append(('scaler', scaler))
        if use_ordinal_encoding:
            if discrete2ord_vars_idx:
                discrete2ord_preprocessor.append(('scaler', scaler))
            if continuous2ord_vars_idx:
                continuous2ord_preprocessor.append(('scaler', scaler))
    if pca_transform_numeric:
        num_preprocessor.append(('scaler', StandardScaler()))
        num_preprocessor.append(('pca', PCA(n_components=len(num_var_names))))

    # Preprocessor part of pipeline
    preprocessor_steps = []
    if num_preprocessor:
        num_transformer = Pipeline(num_preprocessor)
        preprocessor_steps.append(('numeric', num_transformer, num_vars_idx))
    if discrete2ord_preprocessor:
        discrete2ord_transformer = Pipeline(discrete2ord_preprocessor)
        preprocessor_steps.append(('discrete_to_ordinal', discrete2ord_transformer, discrete2ord_vars_idx))
    if continuous2ord_preprocessor:
        continuous2ord_transformer = Pipeline(continuous2ord_preprocessor)
        preprocessor_steps.append(('continuous_to_ordinal', continuous2ord_transformer, continuous2ord_vars_idx))
    if cat_preprocessor:
        cat_transformer = Pipeline(cat_preprocessor)
        preprocessor_steps.append(('categorical', cat_transformer, cat_vars_idx))
    if drop_col_preprocessor:
        drop_col_transformer = Pipeline(drop_col_preprocessor)
        preprocessor_steps.append(('drop_columns', drop_col_transformer, drop_vars_idx))
    preprocessor = ColumnTransformer(transformers=preprocessor_steps, remainder='passthrough')

    # Set random state of classifier
    if 'random_state' in rgr_mdl.get_params():
        rgr_mdl.set_params(random_state=seed_value)
    elif 'random_seed' in rgr_mdl.get_params():
        rgr_mdl.set_params(random_seed=seed_value)

    # Set monotone constraints (only works for gradient-boosting models and linear regression models)
    if set_monotonic_constraints and not pca_transform_numeric:
        monotone_constraints, monotone_columns = monotonic_constraints(
            num_vars=num_var_names,
            ord_vars=discrete2ord_var_names + continuous2ord_var_names, 
            cat_vars=cat_var_names, 
            cat_encode=categorical_encode_method, 
            X=X_train, 
            features_increasing=features_monotone_increasing, 
            features_decreasing=features_monotone_decreasing,
        )
        if mdl_type == 'LGBM':
            rgr_mdl.set_params(monotone_constraints=monotone_constraints, monotone_constraints_method=monotone_constraints_method_lgbm)
        elif mdl_type == 'CatB':
            rgr_mdl.set_params(monotone_constraints=monotone_constraints)
        elif mdl_type == 'HGB':
            rgr_mdl.set_params(monotonic_cst=monotone_constraints)
        elif mdl_type in ['XGB', 'XGB-RF']:
            rgr_mdl.set_params(monotone_constraints=tuple(monotone_constraints))
        elif mdl_type in ['LR', 'Ridge', 'Lasso']:
            rgr_mdl = ConstrainedLinearRegression()
            min_coef = np.repeat(-np.inf, len(monotone_constraints))
            max_coef = np.repeat(np.inf, len(monotone_constraints))
            for j, col in enumerate(monotone_columns):
                if monotone_constraints[j] < 0:
                    max_coef[j] = 0.
                elif monotone_constraints[j] > 0:
                    min_coef[j] = 0.
            if mdl_type == 'Ridge':
                rgr_mdl.set_params(max_coef=max_coef, min_coef=min_coef, ridge=True)
            elif mdl_type == 'Lasso':
                rgr_mdl.set_params(max_coef=max_coef, min_coef=min_coef, lasso=True)
            else:
                rgr_mdl.set_params(max_coef=max_coef, min_coef=min_coef)

    # Transform target variable
    if use_target_transform:
        if target_transform_method == 'yeojohnson':
            rgr_mdl = TransformedTargetRegressor(regressor=rgr_mdl, transformer=YeoJohnsonTargetTransformer(lmbda=target_yeojohnson_lambda, replace_nan_percentile=target_yeojohnson_nan_percentile))
        elif target_transform_method == 'log':
            rgr_mdl = TransformedTargetRegressor(regressor=rgr_mdl, transformer=LogTargetTransformer())

    # Add all steps to final pipeline
    steps = [('preprocessor', preprocessor)] if preprocessor_steps else []
    steps.append(('model', rgr_mdl))
    pipeline = Pipeline(steps=steps)

    return pipeline


# custom function for log-transform
def log_transform(x):
    return np.log1p(x)

def inv_log_transform(x):
    return np.expm1(x)


# Custom function for signed log1p-transform
def signed_log1p_transform(x):
    return np.sign(x) * np.log1p(np.abs(x))

def inv_signed_log1p_transform(x):
    return np.sign(x) * np.expm1(np.abs(x))


# custom function to convert numeric values to strings
def safe_convert_to_str(X):
    return X.applymap(lambda x: str(x) if x is not np.nan else x)


# custom function to drop columns by name
def drop_columns_by_name(X, columns_to_drop, drop=True):
    if drop:
        X = X.drop(columns=columns_to_drop, errors='ignore')
    return X


# custom function to drop rows with ones in y
def drop_rows_with_ones(X, y=None):
    if y is not None:
        mask = y!=1
        return X[mask], y[mask]
    return X


# custom function to drop rows with nans in either X or y
def drop_rows_with_nans(X, y=None):
    nan_mask_X = np.isnan(X).any(axis=1)
    if y is not None:
        nan_mask_y = np.isnan(y)
        nan_mask = nan_mask_X | nan_mask_y
        X_clean = X[~nan_mask]
        y_clean = y[~nan_mask]
        return X_clean, y_clean
    else:
        X_clean = X[~nan_mask_X]
        return X_clean


# function to create monotonic constraints for specific features
def monotonic_constraints(num_vars, ord_vars, cat_vars, cat_encode, X, features_increasing, features_decreasing):
    new_column_order = num_vars
    new_column_order.extend(ord_vars)
    if cat_encode != 'OHE':
        new_column_order.extend(cat_vars)
    else:
        for col in cat_vars:
            ohe = OneHotEncoder(sparse=False)
            ohe.fit_transform(X[[col]])
            new_column_order.extend(ohe.categories_[0])
    monotone_constraints = []
    for col in new_column_order:
        if col in features_increasing:
            monotone_constraints.append(1)
        elif col in features_decreasing:
            monotone_constraints.append(-1)
        else:
            monotone_constraints.append(0)
    return monotone_constraints, new_column_order

### Models

In [ ]:
from model_settings import models

model_for_mapie = 'XGB'
# model_for_mapie = 'LGBM'

print(f"{model_for_mapie}: {models[model_for_mapie]}")

### Sample weights

In [ ]:
def set_sample_weights(y, q=(0.1,0.9), w=2.):
    # get quantiles of the target variable
    if isinstance(q,tuple):
        qmin, qmax = np.max([0.,q[0]*100]), np.min([q[1]*100,100.])
    else:
        qmin, qmax = q*100, 100-(q*100)
    target_quantiles = np.percentile(y, [qmin, qmax])
    # assign weights based on quantiles
    weights = np.zeros_like(y, dtype=float)
    weights[y < target_quantiles[0]] = w  # lower quantile, assign higher weight
    weights[(y >= target_quantiles[0]) & (y <= target_quantiles[1])] = 1.
    weights[y > target_quantiles[1]] = w  # upper quantile, assign higher weight
    return weights

In [ ]:
if use_sample_weights:
    print(set_sample_weights(y_train, q=fit_quantiles_tails, w=fit_weight_tails))

### Base estimator

In [ ]:
from model_settings import target, locations_to_use, frequencies_to_use, columns_not_in_model
from model_settings import imputation_method, scaler_type, scaler_quantile_nquantiles, scaler_rowwise_norm
from model_settings import use_categorical_encoding, use_ordinal_encoding, use_numeric_transform, pca_transform_numeric, use_target_transform
from model_settings import ordinal_columns, continuous_to_ordinal_transform_nbins, categorical_columns, categorical_encode_method, numeric_transform_method
from model_settings import target_transform_method, target_yeojohnson_lambda, target_yeojohnson_nan_percentile
from model_settings import use_sample_weights, fit_weight_tails, fit_quantiles_tails
from model_settings import set_monotonic_constraints, features_monotone_increasing, features_monotone_decreasing, monotone_constraints_method_lgbm
from model_settings import models

# split ordinal features into discrete and continuous
discrete_to_ordinal, continuous_to_ordinal = [], []
for col in ordinal_columns:
    if col not in columns_not_in_model:
        if X_train[col].dtype in ['category', 'object']:
            discrete_to_ordinal.append(col)
        else:
            continuous_to_ordinal.append(col)

# convert X_train columns to list
Xtrain_columns = X_train.columns.tolist()
# set up amodel pipeline
mdl_type = model_for_mapie
rgr_mdl = copy.deepcopy(models[mdl_type].get('model', None))
model = regression_model_pipeline()

# split pipeline in preprocessor and model (works better for mapie 'aci' method)
preprocessor = model.named_steps['preprocessor']
model = model.named_steps['model']
# fit preprocessor before model training
preprocessor.fit(X_train);

In [ ]:
print('Preprocessor:')
display(preprocessor)
print('\nModel:')
display(model)

## Get prediction intervals

In [ ]:
enbpi_cv = False  # if False, EnBPI with BlockBootstrap is not used
aci_cv = False  # if False, ACI with prefit model is used

In [ ]:
mapie_alpha = .1
aci_gamma = 1e-3
print(f"alpha = {mapie_alpha}, gamma = {aci_gamma}")

In [ ]:
locations_for_conformal_cv = ['Europlatform']
locations_for_conformal_prefit = ['Europlatform', 'Eurogeul E13', 'Eurogeul DWE']

In [ ]:
pred_int, cp_metrics = {}, {}

In [ ]:
# if required, use 'prefit' model with interval fit based on calibration set to avoid memory errors when using mapie for large datasets
# see: https://github.com/scikit-learn-contrib/MAPIE/issues/326

In [ ]:
# for source code and implementation of MWIS metric (mean Winkler Interval score) see:  
# https://www.kaggle.com/datasets/carlmcbrideellis/winkler-interval-score-metric  
# https://www.kaggle.com/code/carlmcbrideellis/regression-prediction-intervals-with-mapie

### Set block bootstrap CV

In [ ]:
if (enbpi_cv) | (aci_cv):
    block_length = 50
    n_resamplings = 20
    max_test_size = 30000
    reduce_test_size = False

    if reduce_test_size:
        cv_mapiets = BlockBootstrapWithTestSizeLimit(n_resamplings=n_resamplings, length=block_length, max_test_size=max_test_size, random_state=seed_value)
    else:
        cv_mapiets = BlockBootstrap(n_resamplings=n_resamplings, length=block_length, random_state=seed_value)

### EnbPI conformal prediction

As stated in the MAPIE documentation (https://mapie.readthedocs.io/en/latest/generated/mapie.regression.MapieTimeSeriesRegressor.html), EnbPI only corresponds to MapieTimeSeriesRegressor if the cv argument is of type BlockBootstrap. Since BlockBootstrap with large sample size can produce a memory error, a custom class was created 'BlockBootstrapWithTestSizeLimit' to properly apply EnBPI with a timeseries based cv approach 

EnbPI generates prediction intervals using an ensemble of models and then applies conformal prediction to this ensemble. The idea is to use the ensemble for robust point predictions and then quantify uncertainty by calibrating the intervals based on past performance of the ensemble model. Instead of recalculating intervals at every time point like ACI, EnbPI typically processes data in batches (or windows). It uses the batch predictions from the ensemble model to construct the intervals. These batches can be overlapping or non-overlapping windows, depending on the design.EnbPI does not adapt as fluidly to every individual time step as ACI does but instead aggregates over a batch of predictions.  

EnBPI typically requires block bootstrap or similar approaches as its cross-validation (CV) technique. This is because EnBPI is designed for batch processing and often leverages bootstrap resampling to capture variability in residuals over time. The block bootstrap approach also ensures that the time dependencies in the data (e.g., autocorrelation in time series) are preserved when estimating prediction intervals.

#### EnBPI - block bootstrap

In [ ]:
if enbpi_cv:
    gap = 6

    mdl_cv = copy.deepcopy(model)
    mapie_enbpi = MapieTimeSeriesRegressor(mdl_cv, method="enbpi", cv=cv_mapiets, agg_function="mean")
    print('Fit MAPIE TimeSeriesRegressor with EnBPI')
    mapie_enbpi.fit(preprocessor.transform(X_train.sort_values(['Location', 'Frequency', 'Time'])), y_train.loc[X_train.sort_values(['Location', 'Frequency', 'Time']).index].to_numpy());

    predint_dict = {}
    coverage_dict = {}

    for loc_name in locations_for_conformal_cv:
        print(f"Location: {loc_name}")
        for j,freq in enumerate(X_calib['Frequency'].unique()):
            print(f"  ({j+1}) Frequency: {freq}")
            mapie_enbpi_pfit = copy.deepcopy(mapie_enbpi)

            Xnew_pfit = X_new[(X_new['Location'] == loc_name) & (X_new['Frequency'] == freq)]
            ynew_pfit = y_new[(X_new['Location'] == loc_name) & (X_new['Frequency'] == freq)]
            Xnew_tf = preprocessor.transform(Xnew_pfit)

            y_pred_pfit, y_pis_pfit = np.zeros(len(Xnew_tf)), np.zeros((len(Xnew_tf),2, 1))
            y_pred_pfit[:gap], y_pis_pfit[:gap, :, :] = mapie_enbpi_pfit.predict(
                Xnew_tf[:gap, :], 
                alpha=mapie_alpha, 
                ensemble=True, 
                optimize_beta=True
            )

            for step in tqdm(range(gap, len(Xnew_pfit), gap)):
                mapie_enbpi_pfit.partial_fit(
                    Xnew_tf[(step - gap):step, :], 
                    ynew_pfit.iloc[(step - gap):step].to_numpy()
                )
                y_pred_pfit[step:step + gap], y_pis_pfit[step:step + gap, :, :] = mapie_enbpi_pfit.predict(
                    Xnew_tf[step:(step + gap), :], 
                    alpha=mapie_alpha, 
                    ensemble=True, 
                    optimize_beta=True
                )
                y_pis_pfit[step:step + gap, :, :] = np.clip(
                    y_pis_pfit[step:step + gap, :, :], -10, 10
                )
            predint_dict[f"{loc_name.replace(' ', '-')}_{freq}"] = {
                'y':ynew_pfit.to_numpy(),
                'yhat':y_pred_pfit,
                'q_lower':y_pis_pfit[:,0].flatten(),
                'q_upper':y_pis_pfit[:,1].flatten(),
                'interval':y_pis_pfit[:,1].flatten() - y_pis_pfit[:,0].flatten(),
                'error':y_pred_pfit - ynew_pfit.to_numpy(),
                'freq':Xnew_pfit['Frequency'].values,
                'time':Xnew_pfit['Time'].values,
                'location':Xnew_pfit['Location'].values,
                'index':Xnew_pfit.index.values,
            }
            coverage_dict[f"{loc_name.replace(' ', '-')}_{freq}"] = {
                'coverage':regression_coverage_score(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]),
                'width':regression_mean_width_score(y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]),
                'cwc':coverage_width_based(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0], eta=10, alpha=mapie_alpha),
                'winkler':MWIS_metric.score(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0], alpha=mapie_alpha)[0],
                'freq':freq,
                'location':loc_name,            
            }
        print('')

    pred_int['enbpi'] = pd.DataFrame()
    for key in predint_dict.keys():
        pred_int['enbpi'] = pd.concat([pred_int['enbpi'], pd.DataFrame(predint_dict[key])], axis='index', ignore_index=True)
    pred_int['enbpi'] = pred_int['aci'].set_index('index')
    pred_int['enbpi'].index.name = None

    cp_metrics['enbpi'] = pd.DataFrame()
    for key in predint_dict.keys():
        cp_metrics['enbpi'] = pd.concat([cp_metrics['enbpi'], pd.DataFrame.from_dict(coverage_dict[key], orient='index').transpose()], axis='index', ignore_index=True)
    cp_metrics['enbpi'] = cp_metrics['enbpi'].sort_values(['location','freq']).reset_index(drop=True)

### ACI conformal prediction

ACI updates the width of prediction intervals dynamically, based on the distribution of past residuals (errors in the model’s previous predictions). This approach uses historical data to adjust its intervals over time in an adaptive way, which is important in time series, as the underlying data distribution may change (non-stationarity). The key feature of ACI is that it adapts to changes in the distribution of errors. If the time series becomes more volatile (with larger residuals) or experiences a sudden shift, ACI will produce wider intervals to maintain coverage. Conversely, in more stable periods, it will reduce the width of the intervals.  

ACI does not necessarily require cross-validation or block bootstrap like EnBPI does. Instead, ACI can be applied with either a prefit model or cross-validation. With a prefit model ACI computes prediction intervals sequentially, based on the residuals of the pre-fit model's predictions over time. There’s no need to refit the model during the conformal inference process, and as long as the residuals are updated continuously, the prediction intervals remain valid. ACI can also work with cross-validation to improve generalization but this is not a strict requirement.

#### ACI - block bootstrap

In [ ]:
if aci_cv:
    gap = 6
    
    mdl_cv = copy.deepcopy(model)
    mapie_aci = MapieTimeSeriesRegressor(mdl_cv, method="aci", cv=cv_mapiets, agg_function="mean")
    print('Fit MAPIE TimeSeriesRegressor with ACI')
    mapie_aci.fit(preprocessor.transform(X_train.sort_values(['Location', 'Frequency', 'Time'])), y_train.loc[X_train.sort_values(['Location', 'Frequency', 'Time']).index].to_numpy())

    predint_dict = {}
    coverage_dict = {}

    for loc_name in locations_for_conformal_prediction:
        print(f"Location: {loc_name}")
        for j,freq in enumerate(X_calib['Frequency'].unique()):
            print(f"  ({j+1}) Frequency: {freq}")
            mapie_aci_pfit = copy.deepcopy(mapie_aci)

            Xnew_pfit = X_new[(X_new['Location'] == loc_name) & (X_new['Frequency'] == freq)]
            ynew_pfit = y_new[(X_new['Location'] == loc_name) & (X_new['Frequency'] == freq)]
            Xnew_tf = preprocessor.transform(Xnew_pfit)

            y_pred_pfit, y_pis_pfit = np.zeros(len(Xnew_tf)), np.zeros((len(Xnew_tf),2, 1))
            y_pred_pfit[:gap], y_pis_pfit[:gap, :, :] = mapie_aci_pfit.predict(
                Xnew_tf[:gap, :], 
                alpha=mapie_alpha, 
                ensemble=True, 
                optimize_beta=False,
                allow_infinite_bounds=True,
            )

            for step in tqdm(range(gap, len(Xnew_pfit), gap)):
                mapie_aci_pfit.partial_fit(
                    Xnew_tf[(step - gap):step, :], 
                    ynew_pfit.iloc[(step - gap):step].to_numpy()
                )
                mapie_aci_pfit.adapt_conformal_inference(
                    Xnew_tf[(step - gap):step, :], 
                    ynew_pfit.iloc[(step - gap):step].to_numpy(),
                    gamma=aci_gamma
                )
                y_pred_pfit[step:step + gap], y_pis_pfit[step:step + gap, :, :] = mapie_aci_pfit.predict(
                    Xnew_tf[step:(step + gap), :], 
                    alpha=mapie_alpha, 
                    ensemble=True, 
                    optimize_beta=False,
                    allow_infinite_bounds=True
                )
                y_pis_pfit[step:step + gap, :, :] = np.clip(
                    y_pis_pfit[step:step + gap, :, :], -10, 10
                )
            predint_dict[f"{loc_name.replace(' ', '-')}_{freq}"] = {
                'y':ynew_pfit.to_numpy(),
                'yhat':y_pred_pfit,
                'q_lower':y_pis_pfit[:,0].flatten(),
                'q_upper':y_pis_pfit[:,1].flatten(),
                'interval':y_pis_pfit[:,1].flatten() - y_pis_pfit[:,0].flatten(),
                'error':y_pred_pfit - ynew_pfit.to_numpy(),
                'freq':Xnew_pfit['Frequency'].values,
                'time':Xnew_pfit['Time'].values,
                'location':Xnew_pfit['Location'].values,
                'index':Xnew_pfit.index.values,
            }
            coverage_dict[f"{loc_name.replace(' ', '-')}_{freq}"] = {
                'coverage':regression_coverage_score(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]),
                'width':regression_mean_width_score(y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]),
                'cwc':coverage_width_based(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0], eta=10, alpha=mapie_alpha),
                'winkler':MWIS_metric.score(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0], alpha=mapie_alpha)[0],
                'freq':freq,
                'location':loc_name,            
            }
        print('')

    pred_int['aci'] = pd.DataFrame()
    for key in predint_dict.keys():
        pred_int['aci'] = pd.concat([pred_int['aci'], pd.DataFrame(predint_dict[key])], axis='index', ignore_index=True)
    pred_int['aci'] = pred_int['aci'].set_index('index')
    pred_int['aci'].index.name = None

    cp_metrics['aci'] = pd.DataFrame()
    for key in predint_dict.keys():
        cp_metrics['aci'] = pd.concat([cp_metrics['aci'], pd.DataFrame.from_dict(coverage_dict[key], orient='index').transpose()], axis='index', ignore_index=True)
    cp_metrics['aci'] = cp_metrics['aci'].sort_values(['location','freq']).reset_index(drop=True)

#### ACI - prefit

In [ ]:
if not aci_cv:
    gap = 1

    # prefit model
    mdl_prefit = copy.deepcopy(model)
    if use_sample_weights:
        model_hasattr_sample_weight = True
        try:    
            mdl_prefit.fit(preprocessor.transform(X_train), y_train.to_numpy(), model__sample_weight=np.ones(len(y_train)))
        except:
            model_hasattr_sample_weight = False

        if model_hasattr_sample_weight:
            print('   Applying sample weights for model training')
            mdl_prefit.fit(preprocessor.transform(X_train), y_train.to_numpy(), model__sample_weight=set_sample_weights(y_train, q=fit_quantiles_tails, w=fit_weight_tails))
        else:
            mdl_prefit.fit(preprocessor.transform(X_train), y_train.to_numpy())
    else:
        mdl_prefit.fit(preprocessor.transform(X_train), y_train.to_numpy())

    mapie_aci = MapieTimeSeriesRegressor(mdl_prefit, method="aci", cv="prefit")

    predint_dict = {}
    coverage_dict = {}

    for loc_name in locations_for_conformal_prefit:
        print(f"Location: {loc_name}")
        for j,freq in enumerate(X_calib['Frequency'].unique()):
            print(f"   ({j+1}/{len(X_calib['Frequency'].unique())}) Frequency: {freq}")
            Xcalib_tf = preprocessor.transform(X_calib[(X_calib['Location'] == loc_name) & (X_calib['Frequency'] == freq)])
            mapie_aci_pfit = copy.deepcopy(mapie_aci).fit(Xcalib_tf, y_calib[(X_calib['Location'] == loc_name) & (X_calib['Frequency'] == freq)].to_numpy())

            Xnew_pfit = X_new[(X_new['Location'] == loc_name) & (X_new['Frequency'] == freq)]
            ynew_pfit = y_new[(X_new['Location'] == loc_name) & (X_new['Frequency'] == freq)]
            Xnew_tf = preprocessor.transform(Xnew_pfit)

            y_pred_pfit, y_pis_pfit = np.zeros(len(Xnew_tf)), np.zeros((len(Xnew_tf),2, 1))
            y_pred_pfit[:gap], y_pis_pfit[:gap, :, :] = mapie_aci_pfit.predict(
                Xnew_tf[:gap, :], 
                alpha=mapie_alpha, 
                ensemble=False, 
                optimize_beta=False,
                allow_infinite_bounds=True
            )
            for step in tqdm(range(gap, len(Xnew_pfit), gap)):
                mapie_aci_pfit.partial_fit(
                    Xnew_tf[(step - gap):step, :], 
                    ynew_pfit.iloc[(step - gap):step].to_numpy()
                )
                mapie_aci_pfit.adapt_conformal_inference(
                    Xnew_tf[(step - gap):step, :], 
                    ynew_pfit.iloc[(step - gap):step].to_numpy(),
                    gamma=aci_gamma
                )
                y_pred_pfit[step:step + gap], y_pis_pfit[step:step + gap, :, :] = mapie_aci_pfit.predict(
                    Xnew_tf[step:(step + gap), :], 
                    alpha=mapie_alpha, 
                    ensemble=False, 
                    optimize_beta=False,
                    allow_infinite_bounds=True
                )
                y_pis_pfit[step:step + gap, :, :] = np.clip(
                    y_pis_pfit[step:step + gap, :, :], -10, 10
                )
            predint_dict[f"{loc_name.replace(' ', '-')}_{freq}"] = {
                'y':ynew_pfit.to_numpy(),
                'yhat':y_pred_pfit,
                'q_lower':y_pis_pfit[:,0].flatten(),
                'q_upper':y_pis_pfit[:,1].flatten(),
                'interval':y_pis_pfit[:,1].flatten() - y_pis_pfit[:,0].flatten(),
                'error':y_pred_pfit - ynew_pfit.to_numpy(),
                'freq':Xnew_pfit['Frequency'].values,
                'time':Xnew_pfit['Time'].values,
                'location':Xnew_pfit['Location'].values,
                'index':Xnew_pfit.index.values,
            }
            coverage_dict[f"{loc_name.replace(' ', '-')}_{freq}"] = {
                'coverage':regression_coverage_score(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]),
                'width':regression_mean_width_score(y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]),
                'cwc':coverage_width_based(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0], eta=10, alpha=mapie_alpha),
                'winkler':MWIS_metric.score(ynew_pfit.to_numpy(), y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0], alpha=mapie_alpha)[0],
                'freq':freq,
                'location':loc_name,            
            }
        print('')

    pred_int['aci'] = pd.DataFrame()
    for key in predint_dict.keys():
        pred_int['aci'] = pd.concat([pred_int['aci'], pd.DataFrame(predint_dict[key])], axis='index', ignore_index=True)
    pred_int['aci'] = pred_int['aci'].set_index('index')
    pred_int['aci'].index.name = None

    cp_metrics['aci'] = pd.DataFrame()
    for key in predint_dict.keys():
        cp_metrics['aci'] = pd.concat([cp_metrics['aci'], pd.DataFrame.from_dict(coverage_dict[key], orient='index').transpose()], axis='index', ignore_index=True)
    cp_metrics['aci'] = cp_metrics['aci'].sort_values(['location','freq']).reset_index(drop=True)

## CP results

In [ ]:
if 'enbpi' in pred_int.keys():
    display(pred_int['enbpi'].tail(5))
    display(cp_metrics['enbpi'].tail(5))

In [ ]:
if 'aci' in pred_int.keys():
    display(pred_int['aci'].tail(5))
    display(cp_metrics['aci'].tail(5))

### Plot conformal metrics

In [ ]:
metrics_names = {
    'coverage':'Effective coverage',
    'width':'Effective mean width',
    'winkler':'Winkler Interval score',
}
location_colors = {
    'Europlatform':'tab:blue',
    'Eurogeul E13':'tab:orange',
    'Eurogeul DWE':'tab:green',
}

for cp_method in cp_metrics.keys():
    fig, axes = plt.subplots(figsize=(8,4.5), nrows=3, ncols=1, sharex=True)
    for j,col in enumerate(['width','coverage','winkler']):
        ax = axes.flatten()[j]
        ax.set_title(f"{metrics_names[col]} ({cp_method.replace('enbpi','EnbPI').replace('aci','ACI')})")
        ax.grid(axis='both', color=[.7,.7,.7], linestyle='-', linewidth=.5)
        ax.set_axisbelow(True)
        for loc_name in location_colors.keys():
            if loc_name in cp_metrics[cp_method]['location'].unique():
                ax.plot(
                    cp_metrics[cp_method].loc[cp_metrics[cp_method]['location'] == loc_name, 'freq'], 
                    cp_metrics[cp_method].loc[cp_metrics[cp_method]['location'] == loc_name, col],
                    lw=1., marker='o', markersize=2, color=location_colors[loc_name]
                )

    legend_patches = {}
    for loc_name in location_colors.keys():
        if loc_name in cp_metrics[cp_method]['location'].unique():
            legend_patches[loc_name] = mpatches.Patch(facecolor=location_colors[loc_name], alpha=1., edgecolor='gray', linewidth=.1)
    fig.legend(handles=list(legend_patches.values()), labels=list(legend_patches.keys()), fontsize=10, bbox_to_anchor=(0.98,0.935), loc='upper left')

    plt.tight_layout()
    plt.show()

### Plot prediction intervals

In [ ]:
def plot_prediction_intervals(ax, intervals, freq, location, target, alpha, cp_type, mdl, x_col='rank', sort_by='y', data_to_plot='values', sample_size=1., smoothing_window=None, is_quantile=False, xlim_fixed=None, ylim_fixed=None, signed_logy=False, show_observations=True, show_outliers=False, show_axistitle=True, markersize=5.5):
    plot_df = intervals.copy()
    plot_df = plot_df.loc[(plot_df['freq']==freq) & (plot_df['location']==location)]
    plot_df['mean_interval'] = plot_df['interval'].mean()
    if smoothing_window is not None:
        plot_df['interval'] = plot_df['interval'].rolling(smoothing_window, center=True).mean()
        plot_df['q_lower'] = plot_df['q_lower'].rolling(smoothing_window, center=True).mean()
        plot_df['q_upper'] = plot_df['q_upper'].rolling(smoothing_window, center=True).mean()
    plot_df['outside_interval'] = (plot_df['y'] < plot_df['q_lower']) | (plot_df['y'] > plot_df['q_upper'])
    if signed_logy:
        plot_df['y'] = np.sign(plot_df['y']) * np.log1p(np.abs(plot_df['y']))
        plot_df['yhat'] = np.sign(plot_df['yhat']) * np.log1p(np.abs(plot_df['yhat']))
        plot_df['q_lower'] = np.sign(plot_df['q_lower']) * np.log1p(np.abs(plot_df['q_lower']))
        plot_df['q_upper'] = np.sign(plot_df['q_upper']) * np.log1p(np.abs(plot_df['q_upper']))
        plot_df['interval'] = np.sign(plot_df['interval']) * np.log1p(np.abs(plot_df['interval']))
        plot_df['mean_interval'] = np.sign(plot_df['mean_interval']) * np.log1p(np.abs(plot_df['mean_interval']))
    if x_col == 'time':
        plot_df = plot_df.sort_values('time')
    elif sort_by is not None:
        plot_df = plot_df.sort_values(sort_by)
    plot_df['rank'] = np.cumsum(np.ones(shape=(plot_df.shape[0]),dtype=int))
    plot_df = plot_df.reset_index().sample(n=int(sample_size*plot_df.shape[0])).set_index('index').sort_values('rank')
    if data_to_plot == 'values':
        if show_observations:
            ax.scatter(plot_df[x_col], plot_df['y'], edgecolor='darkorange', facecolor='darkorange', marker='s', s=markersize, lw=.7, label='Observed', zorder=10)
            if show_outliers:
                ax.scatter(
                    plot_df.loc[plot_df['outside_interval'],x_col], plot_df.loc[plot_df['outside_interval'],'y'], 
                    edgecolor='firebrick', facecolor='firebrick', marker='s', s=markersize+1.5, lw=.7, label='Observations outside interval', zorder=30
                )
        ax.scatter(plot_df[x_col], plot_df['yhat'], edgecolor='royalblue', facecolor='royalblue', s=markersize, lw=.7, label='Predicted', zorder=20)
        ax.plot(plot_df[x_col], plot_df['q_lower'], c='gray', ls='-', lw=1., zorder=0)
        ax.plot(plot_df[x_col], plot_df['q_upper'], c='gray', ls='-', lw=1., zorder=0)
        ax.fill_between(plot_df[x_col], plot_df['q_lower'], plot_df['q_upper'], color='gray', label='Prediction interval', alpha=0.3)
    elif data_to_plot == 'errors':
        if x_col == 'rank':
            ax.plot([-20,intervals.shape[0]+20], [0,0], ls='-.', c='firebrick', lw=.7, zorder=20)
        ax.scatter(plot_df[x_col], plot_df['error'], c='royalblue', s=markersize, label='Predicted - Observed', zorder=10)
        ax.plot(plot_df[x_col], plot_df['q_lower'] - plot_df['y'], c='darkorange', ls='--', zorder=0)
        ax.plot(plot_df[x_col], plot_df['q_upper'] - plot_df['y'], c='darkorange', ls='--', zorder=0)
        ax.fill_between(plot_df[x_col], plot_df['q_lower'] - plot_df['y'], plot_df['q_upper'] - plot_df['y'], color='darkorange', label='Prediction interval', alpha=0.4)
    elif data_to_plot == 'intervals':
        ax.plot(plot_df[x_col], plot_df['interval'], c='royalblue', ls='-', lw=1.2, label='Prediction interval', zorder=10)
        xvals = ax.get_xlim()
        ymean = plot_df['mean_interval'].values[0]
        ax.plot([xvals[0],xvals[1]], [ymean,ymean], c='black', ls='--', lw=.7, label='Average interval', zorder=0)
        ax.set_xlim(xvals)
    if x_col == 'time':
        ax.set_xlabel('')
    else:
        ax.set_xlabel(x_col.capitalize())
    if data_to_plot == 'values':
        if signed_logy:
            ax.set_ylabel(f"{target} (signed log)")
        else:
            ax.set_ylabel(target)
    elif data_to_plot == 'errors':
        ax.set_ylabel(f"Predicted ({target}) - Observed ({target})")
    elif data_to_plot == 'intervals':
        ax.set_ylabel(f"Prediction interval ({int(np.round(100*(1-alpha),0))}% coverage)")
    cp_type = cp_type.replace('enbpi','EnbPI').replace('aci','ACI').replace('cqr','Quantile regression')
    if show_axistitle:
        if is_quantile: 
            ax.set_title(f"Location: {location}, Frequency: {freq} - Prediction intervals ({int(np.round(100*(1-alpha),0))}% coverage, {cp_type})\n{str(mdl).split('(')[0]} (conformalized quantile regression)")
        else:
            ax.set_title(f"Location: {location}, Frequency: {freq} - Prediction intervals ({int(np.round(100*(1-alpha),0))}% coverage, {cp_type})\n{str(mdl).split('(')[0]} (conformalized regression)")
    ax.legend(bbox_to_anchor=(1,1.02), loc='upper left')
    if ylim_fixed is not None:
        ax.set_ylim(ylim_fixed)
    elif data_to_plot == 'intervals':
        if plot_df['interval'].max() - plot_df['interval'].min() < 1e-15:
            ax.set_ylim(plot_df['interval'].median()*0.98, plot_df['interval'].median()*1.02)
    
    if xlim_fixed is not None:
        ax.set_xlim(xlim_fixed)
    ax.grid(axis='both', color=[.7,.7,.7], linestyle='-', linewidth=.5)
    ax.set_axisbelow(True)
    return ax

In [ ]:
freq_idx = 16
loc_idx = 2
xlims_plot = pd.to_datetime(['2020-11-05', '2020-11-12'])

In [ ]:
cp_method = 'aci'
smoothing_window = 12

freq_to_plot = X['Frequency'].unique()[freq_idx]
if aci_cv:
    loc_to_plot = locations_for_conformal_cv[0]
else:
    loc_to_plot = locations_for_conformal_prefit[loc_idx]
if split_method == 'leave_one_out':
    loc_to_plot = leave_one_out_location

if cp_method in pred_int.keys():
    fig, ax = plt.subplots(figsize=(16,7), nrows=2, ncols=1, sharex=True, dpi=150)

    ax[0] = plot_prediction_intervals(
        ax[0],
        pred_int[cp_method],
        freq_to_plot,
        loc_to_plot,
        'Correction of SWAN model result', 
        mapie_alpha,
        cp_method,
        mdl=models[mdl_type].get('name'), 
        x_col='time',  # 'rank' or 'time'
        data_to_plot='values', 
        sort_by='time',
        sample_size=1., 
        smoothing_window=smoothing_window, 
        # ylim_fixed=(-.5,1.5),
        # xlim_fixed=xlims_plot,
        # signed_logy=True,
        show_observations=True,
        show_outliers=True,
        markersize=10,
    )
    ax[1] = plot_prediction_intervals(
        ax[1],
        pred_int[cp_method],
        freq_to_plot,
        loc_to_plot,
        'Correction of SWAN model result', 
        mapie_alpha,
        cp_method,
        mdl=models[mdl_type].get('name'), 
        x_col='time',  # 'rank' or 'time'
        data_to_plot='intervals', 
        sort_by='time',
        sample_size=1.,
        smoothing_window=smoothing_window, 
        # ylim_fixed=(0, 2),
        # xlim_fixed=xlims_plot,
        # signed_logy=True,
        show_observations=True,
        show_axistitle=False,
    )
    ax[0].tick_params(axis='x', labeltop=False, labelbottom=True)
    plt.show()

In [ ]:
cp_method = 'enbpi'

freq_to_plot = X['Frequency'].unique()[freq_idx]
loc_to_plot = locations_for_conformal_cv[0]
if split_method == 'leave_one_out':
    loc_to_plot = leave_one_out_location

if cp_method in pred_int.keys():
    fig, ax = plt.subplots(figsize=(16,7), nrows=2, ncols=1, sharex=True, dpi=150)

    ax[0] = plot_prediction_intervals(
        ax[0],
        pred_int[cp_method],
        freq_to_plot,
        loc_to_plot,
        'Correction of SWAN model result', 
        mapie_alpha,
        cp_method,
        mdl=models[mdl_type].get('name'), 
        x_col='time',  # 'rank' or 'time'
        data_to_plot='values', 
        sort_by='time',
        sample_size=1., 
        smoothing_window=None, 
        # ylim_fixed=(-.5,1.5),
        # xlim_fixed=xlims_plot,
        # signed_logy=True,
        show_observations=True,
        show_outliers=True,
        markersize=10,
    )
    ax[1] = plot_prediction_intervals(
        ax[1],
        pred_int[cp_method],
        freq_to_plot,
        loc_to_plot,
        'Correction of SWAN model result', 
        mapie_alpha,
        cp_method,
        mdl=models[mdl_type].get('name'), 
        x_col='time',  # 'rank' or 'time'
        data_to_plot='intervals', 
        sort_by='time',
        sample_size=1.,
        smoothing_window=None, 
        # ylim_fixed=(0, 2),
        # xlim_fixed=xlims_plot,
        # signed_logy=True,
        show_observations=True,
        show_axistitle=False,
    )
    ax[0].tick_params(axis='x', labeltop=False, labelbottom=True)
    plt.show()

## Get feature contribution to prediction intervals

In [ ]:
def shap_kernel_explainer(model, X_trn, y_trn, X_tst, y_tst, sample_method='random', use_testdata=False, sample_size=1000, background_size=20, background_method = 'random', k_kmeans=None, random_seed=42):
    
    # helper function to fix that kernel shap sends data as numpy array without column names
    # see: https://datascience.stackexchange.com/questions/52476/how-to-use-shap-kernal-explainer-with-pipeline-models
    def model_predict(data_asarray):
        data_asframe = pd.DataFrame(data_asarray, columns=fnames)
        return model.predict(data_asframe)
    
    
    # Use test or train data for sample dataset
    if use_testdata:
        X_smp = X_tst.copy()
        y_smp = y_tst.copy()
    else:
        X_smp = X_trn.copy()
        y_smp = y_trn.copy()
    # Use train data as background dataset
    X_bg = X_trn.copy()

    # Take sample from dataset
    if (sample_size is not None):
         if sample_size < X_smp.shape[0]:
            if sample_method == 'random':
                X_smp, _, y_smp, _ = train_test_split(X_smp, y_smp, train_size=min([sample_size, X_smp.shape[0]]), random_state=random_seed)
            elif sample_method == 'stratified':
                groups = create_bins(y_smp, num_bins=10, use_log=True)
                X_smp, _, y_smp, _ = train_test_split(X_smp, y_smp, train_size=min([sample_size, X_smp.shape[0]]), stratify=groups, random_state=random_seed)

    # Take sample from background set
    print(f"   Take sample from background set ({background_method})")
    if background_method == 'kmeans':
        X_bg = shap.kmeans(X_bg, k=k_kmeans)
    else:
        X_bg = X_bg.sample(n=min([background_size, X_bg.shape[0]]), random_state=random_seed).values

    # Explain predicted values
    print('   Create explainer')
    fnames = X_smp.columns.tolist()
    explainer = shap.KernelExplainer(
        model_predict, 
        X_bg,
    )
    # Get SHAP values
    print('   Get SHAP values')
    shap_values = explainer.shap_values(X_smp)

    # Build results dictionary
    shap_res = {}
    shap_res['explainer'] = explainer
    shap_res['shap_values'] = shap_values
    shap_res['shap_feature_names'] = fnames
    shap_res['X'] = X_smp
    shap_res['y'] = y_smp
    shap_res['X_transformed'] = X_smp
    shap_res['y_pred'] = pd.Series(model.predict(X_smp), index=y_smp.index)
    
    return shap_res

In [ ]:
def shap_permutation_explainer(model, X_trn, y_trn, X_tst, y_tst, sample_method='random', use_testdata=False, sample_size=1000, background_size=20, background_method = 'random', k_kmeans=5, ord_feats=[], cat_feats=[], drop_feats=[], random_seed=42):
    
    # define 'column' order in np.array after preprocessing (1. numeric, 2. ordinal, 3. categorical)
    ord_var_names = X_trn.columns[(X_trn.columns.isin(ord_feats)) & (~X_trn.columns.isin(drop_feats))].tolist()
    cat_var_names = X_trn.columns[(X_trn.columns.isin(cat_feats)) & (~X_trn.columns.isin(drop_feats))].tolist()
    num_var_names = X_trn.columns[(~X_trn.columns.isin(cat_var_names)) & (~X_trn.columns.isin(ord_var_names)) & (~X_trn.columns.isin(drop_feats))].tolist()
    new_column_order = num_var_names
    new_column_order.extend(ord_var_names)
    new_column_order.extend(cat_var_names)

    # Use test or train data for sample dataset
    if use_testdata:
        X_smp = X_tst.copy()
        y_smp = y_tst.copy()
    else:
        X_smp = X_trn.copy()
        y_smp = y_trn.copy()
    # Use train data as background dataset
    X_bg = X_trn.copy()
    y_bg = y_trn.copy()

    # print(X_smp.shape[0])
    
    # Take sample from dataset
    if (sample_size is not None):
         if sample_size < X_smp.shape[0]:
            if sample_method == 'random':
                X_smp, _, y_smp, _ = train_test_split(X_smp, y_smp, train_size=min([sample_size, X_smp.shape[0]]), random_state=random_seed)
            elif sample_method == 'stratified':
                groups = create_bins(y_smp, num_bins=10, use_log=True)
                X_smp, _, y_smp, _ = train_test_split(X_smp, y_smp, train_size=min([sample_size, X_smp.shape[0]]), stratify=groups, random_state=random_seed)
    
    # Transform features
    if 'preprocessor' in model.named_steps:
        data_transformer = model.named_steps['preprocessor'].fit(X_trn, y_trn)
        transformer_uses_y = True
        try:
            data_transformer.transform(X_smp, y_smp)
        except:
            transformer_uses_y = False
        if transformer_uses_y:
            X_tf = data_transformer.transform(X_smp, y_smp)
        else:
            X_tf = data_transformer.transform(X_smp)
    else:
        X_tf = X_smp.values

    # Make background dataset
    if background_size is not None:
        X_bg = X_bg.sample(n=min([background_size, X_bg.shape[0]]), random_state=random_seed)
        y_bg = y_bg.sample(n=min([background_size, y_bg.shape[0]]), random_state=random_seed)
    if 'preprocessor' in model.named_steps:
        if transformer_uses_y:
            X_bg = data_transformer.transform(X_bg, y_bg)
        else:
            X_bg = data_transformer.transform(X_bg)

    # Explain predicted values
    print('   Create explainer')
    explainer = shap.PermutationExplainer(
        model.named_steps['model'].predict, 
        X_bg,
    )
    # Get SHAP values
    print('   Get SHAP values')
    shap_values = explainer.shap_values(X_tf)
    # Check if shap_values is array
    if not isinstance(shap_values, np.ndarray):
        shap_values = shap_values.values

    # Aggregate all categories of onehot encoded features
    if use_categorical_encoding:
        # Get number of unique categories for each feature 
        n_categories = []
        for feat in new_column_order[:-1]:
            if feat in cat_feats:
                n = X_smp[feat].nunique()
                n_categories.append(n)
            else:
                n_categories.append(1)
        # Sum SHAP values of all categories in each feature
        new_shap_values = []
        for values in shap_values:
            # Split shap values into a list for each feature
            values_split = np.split(values , np.cumsum(n_categories))
            # Sum values within each list per feature
            values_sum = [sum(l) for l in values_split]
            new_shap_values.append(values_sum)
        new_shap_values = np.array(new_shap_values)
        # Replace SHAP values with new values
        shap_values = new_shap_values

    # Build results dictionary
    shap_res = {}
    shap_res['explainer'] = explainer
    shap_res['shap_values'] = shap_values
    shap_res['shap_feature_names'] = new_column_order
    shap_res['X'] = X_smp
    shap_res['y'] = y_smp
    shap_res['X_transformed'] = X_tf
    shap_res['y_pred'] = pd.Series(model.predict(X_smp), index=y_smp.index)
    
    return shap_res

In [ ]:
def shap_explainer(model, X_trn, y_trn, X_tst, y_tst, sample_method='random', use_testdata=False, fit_preprocessor=True, sample_size=1000, background_size=20, feature_perturbation='interventional', ord_feats=[], cat_feats=[], drop_feats=[], random_seed=42):
# https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/linear_models/Explaining%20a%20model%20that%20uses%20standardized%20features.html
# Any univariate transformation applied to a model’s inputs does not effect the Shapley values for the model. Note that 
# multi-variate transformations like PCA decompositions do change the Shapley values so this trick does not apply there.
    
    # define 'column' order in np.array after preprocessing (1. numeric, 2. ordinal, 3. categorical)
    ord_var_names = X_trn.columns[(X_trn.columns.isin(ord_feats)) & (~X_trn.columns.isin(drop_feats))].tolist()
    cat_var_names = X_trn.columns[(X_trn.columns.isin(cat_feats)) & (~X_trn.columns.isin(drop_feats))].tolist()
    num_var_names = X_trn.columns[(~X_trn.columns.isin(cat_var_names)) & (~X_trn.columns.isin(ord_var_names)) & (~X_trn.columns.isin(drop_feats))].tolist()
    new_column_order = num_var_names
    new_column_order.extend(ord_var_names)
    new_column_order.extend(cat_var_names)
    
    # Use test or train data for sample dataset
    if use_testdata:
        X_smp = X_tst.copy()
        y_smp = y_tst.copy()
    else:
        X_smp = X_trn.copy()
        y_smp = y_trn.copy()
    set_size = y_smp.shape[0]
    # Use train data as background dataset
    X_bg = X_trn.copy()
    y_bg = y_trn.copy()

    # Take sample from dataset
    if (sample_size is not None):
         if sample_size < X_smp.shape[0]:
            if sample_method == 'random':
                X_smp, _, y_smp, _ = train_test_split(X_smp, y_smp, train_size=min([sample_size, X_smp.shape[0]]), random_state=random_seed)
            elif sample_method == 'stratified':
                groups = create_bins(y_smp, num_bins=10, use_log=True)
                X_smp, _, y_smp, _ = train_test_split(X_smp, y_smp, train_size=min([sample_size, X_smp.shape[0]]), stratify=groups, random_state=random_seed)

    # Transform features
    if 'preprocessor' in model.named_steps:
        data_transformer = model.named_steps['preprocessor']
        if fit_preprocessor:
            data_transformer.fit(X_trn, y_trn)
        transformer_uses_y = True
        try:
            data_transformer.transform(X_smp, y_smp)
        except:
            transformer_uses_y = False
        if transformer_uses_y:
            X_tf = data_transformer.transform(X_smp, y_smp)
        else:
            X_tf = data_transformer.transform(X_smp)
    else:
        X_tf = X_smp.values
    # print(f"   Column order of transformed data:\n   {new_column_order}")

    # Make background dataset
    if background_size is not None:
        X_bg = X_bg.sample(n=min([background_size, X_bg.shape[0]]), random_state=random_seed)
        y_bg = y_bg.sample(n=min([background_size, y_bg.shape[0]]), random_state=random_seed)
    if 'preprocessor' in model.named_steps:
        if transformer_uses_y:
            X_bg = data_transformer.transform(X_bg, y_bg)
        else:
            X_bg = data_transformer.transform(X_bg)

    # Check type of classifier
    is_linear_classifier = True
    try:
        shap.LinearExplainer(model.named_steps['model'], X_bg)
    except:
        is_linear_classifier = False

    # Explain predicted values
    if not is_linear_classifier: 
        print('   Create TreeExplainer')
        explainer = shap.TreeExplainer(
            model.named_steps['model'], 
            X_bg if feature_perturbation=='interventional' else None,
            feature_perturbation=feature_perturbation if feature_perturbation=='interventional' else 'correlation_dependent',
        )
    
    elif is_linear_classifier:
        print('   Create LinearExplainer')
        explainer = shap.LinearExplainer(
            model.named_steps['model'], 
            X_bg,
            feature_perturbation=None if feature_perturbation=='interventional' else 'correlation_dependent',
        )
    # Get SHAP values
    print('   Get SHAP values')
    if 'assert_additivity' in dir(explainer):
        shap_values = explainer(X_tf, check_additivity=False)
    else:
        shap_values = explainer(X_tf)
    # Check if shap_values is array
    if not isinstance(shap_values, np.ndarray):
        shap_values = shap_values.values
    # Get interaction values
    if (feature_perturbation!='interventional') & (not is_linear_classifier):
        interaction_values = explainer.shap_interaction_values(X_tf)
    else:
        interaction_values = None

    # Aggregate all categories of onehot encoded features
    if use_categorical_encoding:
        # Get number of unique categories for each feature 
        n_categories = []
        for feat in new_column_order[:-1]:
            if feat in cat_feats:
                n = X_smp[feat].nunique()
                n_categories.append(n)
            else:
                n_categories.append(1)
        
        # Sum SHAP values of all categories in each feature
        new_shap_values = []
        for values in shap_values:
            # Split shap values into list for each feature
            values_split = np.split(values , np.cumsum(n_categories))
            # Sum values within each list per feature
            values_sum = [sum(l) for l in values_split]
            new_shap_values.append(values_sum)
        new_shap_values = np.array(new_shap_values)
        
        # Replace SHAP values with new values
        shap_values = new_shap_values

        # Apply vectorized operations to sum interaction values if they exist
        if interaction_values is not None:
            # Get number of unique categories for each feature
            n_categories = []
            for feat in new_column_order:
                if feat in cat_feats:
                    n = X_smp[feat].nunique()
                    n_categories.append(n)
                else:
                    n_categories.append(1)

            n_samples, _, _ = interaction_values.shape
            n_agg_features = len(n_categories)
            
            # Create indexing arrays
            cumsum_categories = np.cumsum([0] + n_categories)
            
            # Create new array for aggregated interaction values
            new_interaction_values = np.zeros((n_samples, n_agg_features, n_agg_features))
            
            # Use np.add.at for efficient aggregation
            for i in range(n_agg_features):
                for j in range(n_agg_features):
                    np.add.at(new_interaction_values[:, i, j],
                              (),
                              interaction_values[:, cumsum_categories[i]:cumsum_categories[i+1],
                                                    cumsum_categories[j]:cumsum_categories[j+1]].sum(axis=(1,2)))
            
            # Replace SHAP interaction values with new values
            interaction_values = new_interaction_values
    
    # Build results dictionary
    shap_res = {}
    shap_res['explainer'] = explainer
    shap_res['shap_values'] = shap_values
    if interaction_values is not None:
        shap_res['interaction_values'] = interaction_values
    shap_res['shap_feature_names'] = new_column_order
    shap_res['X'] = X_smp
    shap_res['y'] = y_smp
    shap_res['X_transformed'] = X_tf
    shap_res['y_pred'] = pd.Series(model.predict(X_smp), index=y_smp.index)

    return shap_res

### Set parameters for SHAP

In [ ]:
cp_method = 'aci'

shap_size = None
shap_background_size = 100

shap_sample_method = 'stratified'
# shap_sample_method = 'random'

# shap_feature_perturbation = 'interventional'
shap_feature_perturbation = 'not interventional'

method_for_invalid_classifiers = 'permutation'
# method_for_invalid_classifiers = 'kernel'

if pca_transform_numeric:
    method_for_invalid_classifiers = 'kernel'

cp_target_diff = False
# cp_target_diff = True

In [ ]:
data_cp = X_new.copy().join(pred_int[cp_method]['interval'], how='left').rename(columns={'interval':'Conformal interval'})
data_cp = data_cp.sort_values(['Location','Frequency','Time'])
if cp_target_diff:
    data_cp['Conformal interval'] = data_cp.groupby(['Location','Frequency'])['Conformal interval'].transform(lambda x: x.diff()).fillna(0)

display(data_cp.head())

### Calculate SHAP values per location-frequency combination

In [ ]:
columns_not_in_model_ = columns_not_in_model.copy()
columns_not_in_model.extend(['Location','Frequency'])
columns_not_in_model.extend(data_cp.filter(regex='lag=|MA=').columns)
columns_not_in_model_cp = columns_not_in_model.copy()

use_target_transform_ = use_target_transform
use_target_transform = False

rgr_mdl = XGBRegressor(objective='reg:squarederror', n_estimators=300, random_state=seed_value)
# rgr_mdl = LGBMRegressor(n_estimators=50, force_col_wise=True, verbose=0, random_state=seed_value)
# rgr_mdl = LinearRegression()

# convert X_train columns to a list
Xtrain_columns = data_cp.drop(columns=['Conformal interval']).columns.tolist()
# set model pipeline
pipe_cp = regression_model_pipeline()

shap_values_cp = {}

for loc_name in locations_for_conformal_prefit:
    print(f"Location: {loc_name}")
    shap_values_cp[loc_name] = {}
    for i,freq in enumerate(X['Frequency'].sort_values().unique()):
        print(f" ({i+1}/{len(X['Frequency'].unique())}) Frequency: {freq}")
        X_cp = data_cp.copy().loc[(data_cp['Location'] == loc_name) & (data_cp['Frequency'] == freq)].drop(columns='Conformal interval')
        y_cp = data_cp.copy().loc[(data_cp['Location'] == loc_name) & (data_cp['Frequency'] == freq),'Conformal interval']
        
        # split pipeline in preprocessor and model (works better for mapie 'aci' method)
        preprocessor_cp = pipe_cp.named_steps['preprocessor']
        model_cp = copy.deepcopy(pipe_cp.named_steps['model'])
        # fit preprocessor before model training
        preprocessor_cp.fit(X_train, y_train)
        model_cp.fit(preprocessor_cp.transform(X_cp), y_cp)
        pipe_cp_ = Pipeline(steps=[('preprocessor', preprocessor_cp), ('model', model_cp)])

        # check if classifier works with the SHAP Explainer class
        is_valid_shap_classifier = True
        try:
            shap.Explainer(
                pipe_cp_.named_steps['model'], 
                pipe_cp_.named_steps['preprocessor'].fit_transform(X_cp, y_cp),
            )
        except:
            is_valid_shap_classifier = False

        if pca_transform_numeric:
            is_valid_shap_classifier = False

        # estimate SHAP values
        if is_valid_shap_classifier:
            print('   Uses SHAP Explainer')
            shap_values_cp[loc_name][freq] = shap_explainer(
                pipe_cp_, 
                X_trn=X_cp, 
                y_trn=y_cp,
                X_tst=None,
                y_tst=None,
                use_testdata=False,
                fit_preprocessor=False,
                sample_method=shap_sample_method,
                sample_size=shap_size,
                background_size=shap_background_size,
                feature_perturbation=shap_feature_perturbation,
                cat_feats=categorical_columns if use_categorical_encoding else [None],
                ord_feats=(discrete_to_ordinal + continuous_to_ordinal) if use_ordinal_encoding else [None],
                drop_feats=columns_not_in_model_cp,
                random_seed=seed_value,
            )
        else:
            if method_for_invalid_classifiers == 'permutation':
                print('   Uses SHAP Permutation Explainer')
                shap_values_cp[loc_name][freq] = shap_permutation_explainer(
                    pipe_cp_, 
                    X_trn=X_cp, 
                    y_trn=y_cp,
                    X_tst=None,
                    y_tst=None,
                    use_testdata=False,
                    sample_method=shap_sample_method,
                    sample_size=shap_size,
                    background_size=shap_background_size,
                    cat_feats=categorical_columns if use_categorical_encoding else [None],
                    ord_feats=(discrete_to_ordinal + continuous_to_ordinal) if use_ordinal_encoding else [None],
                    drop_feats=columns_not_in_model_cp,
                    random_seed=seed_value,
                )
            else:
                print('   Uses SHAP Kernel Explainer')
                shap_values_cp[loc_name][freq] = shap_kernel_explainer(
                    pipe_cp_, 
                    X_trn=X_cp, 
                    y_trn=y_cp,
                    X_tst=None,
                    y_tst=None,
                    use_testdata=False,
                    sample_method=shap_sample_method,
                    sample_size=shap_size,
                    background_size=shap_background_size,
                    k_kmeans=None,
                    random_seed=seed_value,
                )

        shap_values_cp[loc_name][freq]['Time'] = X_cp.loc[shap_values_cp[loc_name][freq]['X'].index,'Time'].values
        shap_values_cp[loc_name][freq]['Location'] = X_cp.loc[shap_values_cp[loc_name][freq]['X'].index,'Location'].values
        shap_values_cp[loc_name][freq]['Frequency'] = X_cp.loc[shap_values_cp[loc_name][freq]['X'].index,'Frequency'].values

use_target_transform = use_target_transform_
columns_not_in_model = columns_not_in_model_

### Plot SHAP of features over time for conformal intervals

In [ ]:
def plot_shap_over_time(shap_data, freq, loc_name, win_size=6, max_features=10, cmap='tab10', skip_neg=False, color_other='lightgray', sort_statistic='max', show_features=True, cp_method='aci'):
    """
    Function to plot SHAP values over time with the top N SHAP features stacked
    and actual vs predicted values in a separate axis.
    
    Parameters:
    - shap_values_cp: A dictionary containing SHAP values and other related data
    - freq: The frequency to plot (index into the frequency dimension)
    - loc_name: The location to plot (key to select the location from shap_values_cp)
    - win_size: The size of the rolling window to smooth the SHAP values (default: 6)
    - max_features: The maximum number of top SHAP features to display (default: 10)
    - cmap: The colormap to use for feature plots (default: 'tab10')
    - color_other: Color for the 'Other' category (default: light gray)
    - sort_type: Type of sorting to use for the top features (default: 'max')
    - show_features: Whether to show the features in a separate axis (default: True)
    - cp_method: The method to used for calculating the prediction intervals (default: 'aci')
    """
    
    # Create DataFrame for SHAP values
    shap_to_plot = pd.DataFrame(shap_data[loc_name][freq]['shap_values'], 
                                columns=shap_data[loc_name][freq]['shap_feature_names'])
    shap_to_plot = shap_to_plot.assign(
        **{
            'Time': shap_data[loc_name][freq]['Time'],
        }
    )

    # Step 1: Calculate mean absolute SHAP values for each feature (sorted in descending order)
    if sort_statistic == 'median':
        sorted_shap_values = shap_to_plot.drop(columns=['Time']).abs().agg(np.median).sort_values(ascending=False)
    else:
        sorted_shap_values = shap_to_plot.drop(columns=['Time']).abs().agg(sort_statistic).sort_values(ascending=False)

    # Step 2: Select top N features based on mean SHAP values
    top_features = sorted_shap_values.index[:max_features].tolist()
    other_features = sorted_shap_values.index[max_features:].tolist()
    if len(other_features) == 1:
        top_features.extend(other_features)
        other_features = []

    # Sort top features in ascending order by mean SHAP value
    sorted_top_features = sorted_shap_values[top_features].sort_values().index

    # Step 3: Aggregate other features into an 'Other' category
    if len(other_features) > 0:
        shap_to_plot['Other features'] = shap_to_plot[other_features].sum(axis='columns')

    # Step 4: Create a new dataframe with only top N features and 'Other'
    if len(other_features) > 0:
        shap_top_N = shap_to_plot[['Time'] + ['Other features'] + list(sorted_top_features)]  # Sorted top features
    else:
        shap_top_N = shap_to_plot[['Time'] + list(sorted_top_features)]  # Sorted top features

    # Step 5: Apply rolling window smoothing **before** splitting into positive and negative
    for feature in shap_top_N.drop(columns='Time', errors='ignore').columns:  # Skip 'Time' column
        shap_top_N.loc[:, feature] = shap_top_N[feature].rolling(window=win_size, center=True).mean()
    
    # Step 6: Split into positive and negative SHAP values **after rolling**
    shap_pos = shap_top_N.copy()
    shap_neg = shap_top_N.copy()

    # Replace negative SHAP values with 0 in the positive dataframe, and vice versa
    for feature in shap_top_N.drop(columns='Time', errors='ignore').columns:  # Skip 'Time' column
        shap_pos[feature] = shap_top_N[feature].apply(lambda x: x if x > 0 else 0)
        shap_neg[feature] = shap_top_N[feature].apply(lambda x: x if x < 0 else 0)

    # Step 7: Define the colormap
    color_map = plt.get_cmap(cmap)
    num_features = len(shap_top_N.drop(columns='Time', errors='ignore').columns)  # Number of features to plot
    if len(other_features) > 0:
        colors = [color_other] + [color_map(i) for i in range(num_features - 1)][::-1]
    else:
        colors = [color_map(i) for i in range(num_features)][::-1]

    # Step 8: Plot SHAP stacked area chart (Second Axis)
    fig, ax = plt.subplots(nrows=3 if show_features else 2, ncols=1, sharex=True, figsize=(16,16) if show_features else (16, 12), dpi=150)
    if show_features:
        fig.subplots_adjust(hspace=0.4)
    else:
        fig.subplots_adjust(hspace=0.2)

    # Initialize baselines for stacking (positive and negative)
    pos_baseline = np.zeros(len(shap_pos['Time']))
    neg_baseline = np.zeros(len(shap_neg['Time']))
    # Add each feature to the plot
    for i, feature in enumerate(shap_top_N.drop(columns='Time', errors='ignore').columns):  # Skip 'Time' column
        # Stack positive SHAP values (upwards)
        ax[0].fill_between(shap_pos['Time'], 
                           pos_baseline,  # Previous cumulative baseline
                           pos_baseline + shap_pos[feature],  # New cumulative value
                           label=feature, color=colors[i],
                           linewidth=0.05)
        # Update positive baseline
        pos_baseline += shap_pos[feature]
        
        # Stack negative SHAP values (downwards)
        if not skip_neg:
            ax[0].fill_between(shap_neg['Time'], 
                               neg_baseline,  # Previous cumulative baseline
                               neg_baseline + shap_neg[feature],  # New cumulative value
                               label=None, color=colors[i],
                               linewidth=0.05)
            # Update negative baseline
            neg_baseline += shap_neg[feature]

    # Customize first axis
    ax[0].grid(axis='both', color=[.7, .7, .7], linestyle='-', linewidth=.5)
    ax[0].set_axisbelow(True)
    ylabel = 'SHAP-based Feature Contributions\nto Prediction Intervals'
    if win_size > 1:
        ylabel = ylabel + f" (MA, window={win_size}hr)"
    ax[0].set_ylabel(ylabel, fontsize=12)

    ax[0].set_ylabel(ylabel, fontsize=12)
    ax[0].set_title(f"Location: '{loc_name}', Frequency: {freq}", fontsize=22)
    ax[0].tick_params(axis='x', labeltop=False, labelbottom=True)

    # Flip entire legend
    handles, labels = ax[0].get_legend_handles_labels()
    ax[0].legend(handles[::-1], labels[::-1], loc='upper left', bbox_to_anchor=(1., 1.02), ncol=1, fontsize=9)

    # Step 9: Plot predicted vs actual values (First Axis)
    ax[1].plot(shap_data[loc_name][freq]['Time'], 
               shap_data[loc_name][freq]['y'].rolling(window=win_size, center=True).mean(), 
               lw=2.5, label=f"{cp_method.upper()} Interval Estimate", zorder=0)
    ax[1].plot(shap_data[loc_name][freq]['Time'], 
               shap_data[loc_name][freq]['y_pred'].rolling(window=win_size, center=True).mean(), 
               lw=1.5, label="Model-based Reconstruction", zorder=10)
    ax[1].grid(axis='both', color=[.7, .7, .7], linestyle='-', linewidth=.5)
    ax[1].set_axisbelow(True)
    ax[1].set_ylabel(f"Estimated Prediction Intervals using {cp_method.upper()} Method\nwith Step-wise Update of Residuals Distribution", fontsize=12)
    ax[1].legend(loc='upper left', bbox_to_anchor=(1., 1.02), ncol=1, fontsize=9)
    ax[1].tick_params(axis='x', labeltop=False, labelbottom=True)

    # Step 10: Show top N features
    if show_features:
        if len(other_features) > 0:
            colors = colors[1:]
        colors = colors[::-1]
        for i,feature in enumerate(top_features):
            ax[2].plot(shap_data[loc_name][freq]['X']['Time'], shap_data[loc_name][freq]['X'][feature] / shap_data[loc_name][freq]['X'][feature].abs().max(), lw=1.2, label=feature, color=colors[i])
        ax[2].grid(axis='both', color=[.7, .7, .7], linestyle='-', linewidth=.5)
        ax[2].set_axisbelow(True)
        ax[2].set_ylabel('Normalized Feature Values', fontsize=12)
        ax[2].legend(loc='upper left', bbox_to_anchor=(1., 1.02), ncol=1, fontsize=9)

    plt.show()

In [ ]:
loc_idx = 2
freq_idx = 16
win_size = 48
max_features = 99

freq = X['Frequency'].unique()[freq_idx]
loc_name = locations_for_conformal_prefit[loc_idx]
plot_shap_over_time(
    shap_values_cp, 
    freq=freq, 
    loc_name=loc_name, 
    cmap='tab20', 
    win_size=win_size, 
    max_features=max_features, 
    color_other='lightgray',
    sort_statistic='median',
    show_features=False,
    cp_method=cp_method,
)

In [ ]:
if 'interaction_values' in shap_values_cp[loc_name][freq]:
    shap.summary_plot(
        shap_values_cp[loc_name][freq]['interaction_values'], 
        pd.DataFrame(shap_values_cp[loc_name][freq]['X'], columns=shap_values_cp[loc_name][freq]['shap_feature_names']), 
        max_display=len(shap_values_cp[loc_name][freq]['shap_feature_names'])
    )

## Get treatment effects of features for prediction intervals

### Check collinearity

In [ ]:
def check_collinearity(dataf, feats_inmodel, X_coll=[], use_statsmodels=False):
    X_vif = dataf.copy()[feats_inmodel]
    X_vif = X_vif.drop(columns=X_coll, errors='ignore').dropna(axis='index')
    cols_categorical = X_vif.columns[X_vif.dtypes=='category']
    X_vif[cols_categorical] = X_vif[cols_categorical].apply(lambda x: x.cat.codes)
    if use_statsmodels:
        # VIF calculation with variance_inflation_factor function from statsmodels
        # variance_inflation_factor expects the presence of a constant in the matrix of explanatory variables 
        # (https://stackoverflow.com/questions/42658379/variance-inflation-factor-in-python)
        X_vif['Intercept'] = 1.
        vif_info = pd.DataFrame()
        vif_info = vif_info.assign(
            **{
                'Features':X_vif.columns,
                'VIF':[variance_inflation_factor(X_vif.values, i) for i in range(X_vif.shape[1])],
            }
        ).sort_values('VIF', ascending=False)
        vif_info = vif_info[vif_info['Features']!='Intercept']
    else:
        # VIF calculation without variance_inflation_factor function from statsmodels
        vif_info = np.diag(np.linalg.inv(np.corrcoef(X_vif.values, rowvar=False)))
        vif_info = pd.DataFrame(vif_info, index=X_vif.columns).reset_index().rename(columns={'index':'Features',0:'VIF'})
        vif_info = vif_info.sort_values('VIF', ascending=False)
    return vif_info

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score

def feature_variance_explained(X, y=None, cols_to_drop=[], add_target=False):
    X_var = X.copy().drop(columns=cols_to_drop, errors='ignore')
    if add_target & (y is not None):
        X_var = X_var.assign(**{f"{target} (=target)":y})

    label_encoder = LabelEncoder()
    for col in X_var.columns:
        if X_var[col].apply(lambda x: isinstance(x, str)).all():
            X_var[col] = label_encoder.fit_transform(X_var[col])
    
    var_explained = {}
    for feat_name in X_var.columns:
        var_explained[feat_name] = {}
        for col in X_var.columns:
            mdl = LinearRegression().fit(X_var[[col]].values, X_var[feat_name].values)
            var_explained[feat_name][col] = np.sqrt(explained_variance_score(X_var[feat_name].values, mdl.predict(X_var[col].values.reshape(-1,1))))
    
    var_explained = pd.DataFrame(var_explained)
    np.fill_diagonal(var_explained.values, np.nan)
    
    return var_explained

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

columns_not_in_model_ = columns_not_in_model.copy()
columns_not_in_model.extend(['Location','Frequency','Conformal interval'])
columns_not_in_model.extend(data_cp.filter(regex='lag=|MA=').columns)
columns_not_in_model_te = columns_not_in_model.copy()

vif_info = check_collinearity(data_cp, data_cp.columns, X_coll=columns_not_in_model_te, use_statsmodels=False)
display(vif_info)

explained_var = feature_variance_explained(data_cp, y=data_cp['Conformal interval'], cols_to_drop=columns_not_in_model, add_target=False)
fig, ax = plt.subplots(figsize=(6,6))
fig.subplots_adjust(bottom=0.05, top=0.92, left=0.1, right=0.9)
fig.suptitle('Feature Variance Explained', fontsize=16)
sns.heatmap(
    explained_var, 
    ax=ax, 
    vmin=0, 
    vmax=1, 
    annot=True, 
    fmt='.3f', 
    cmap='Blues', 
    cbar=False, 
    linewidths=.5, 
    linecolor='gray', 
    annot_kws={'fontsize':8}
)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='right')
spine_color = 'gray'
for spine in ['top', 'right', 'bottom', 'left']:
    ax.spines[spine].set_visible(True)
    ax.spines[spine].set_color(spine_color)
    ax.spines[spine].set_linewidth(.7)
plt.show()

columns_not_in_model = columns_not_in_model_

### Set parameters for DML

In [ ]:
# use_kbins = False
use_kbins = True

nbins = 100

if use_kbins:
    cv_folds = 5
else:
    cv_folds = 1

dml_type = 'linear'
# dml_type = 'forest'
# dml_type = 'orthogonal'

# cv_sampling = 'random'
cv_sampling = 'stratified'

cv_nbins = 5

### Calculate individual treatment effects per location-frequency combination

In [ ]:
columns_not_in_model_ = columns_not_in_model.copy()
columns_not_in_model.extend(['Location','Frequency'])
columns_not_in_model.extend(data_cp.filter(regex='lag=|MA=').columns)
columns_not_in_model_te = columns_not_in_model.copy()

use_target_transform_ = use_target_transform
use_target_transform = False

rgr_mdl = XGBRegressor(objective='reg:squarederror', n_estimators=300, random_state=seed_value)
# rgr_mdl = LGBMRegressor(n_estimators=50, force_col_wise=True, verbose=0, random_state=seed_value)
# rgr_mdl = LinearRegression()

# convert X_train columns to a list
Xtrain_columns = data_cp.drop(columns=['Conformal interval']).columns.tolist()
# set model pipeline
pipe_te = regression_model_pipeline()

ite_values_cp = {}

for loc_name in locations_for_conformal_prefit:
    print(f"Location: {loc_name}")
    ite_values_cp[loc_name] = {}
    for i,freq in enumerate(X['Frequency'].sort_values().unique()):
        ite_values_cp[loc_name][freq] = {}
        print(f"  ({i+1}/{len(X['Frequency'].unique())}) Frequency: {freq}")
        X_te = data_cp.copy().loc[(data_cp['Location'] == loc_name) & (data_cp['Frequency'] == freq)].drop(columns='Conformal interval')
        y_te = data_cp.copy().loc[(data_cp['Location'] == loc_name) & (data_cp['Frequency'] == freq),'Conformal interval']
        
        # split pipeline in preprocessor and model
        preprocessor_te = pipe_te.named_steps['preprocessor']
        outcome_model = pipe_te.named_steps['model']
        # fit preprocessor before model training
        preprocessor_te.fit(X_train, y_train);
        X_te_tf = pd.DataFrame(preprocessor_te.transform(X_te), columns=X_te.drop(columns=columns_not_in_model_te).columns)
        # fit outcome model
        outcome_model.fit(X_te_tf, y_te)
        # build treatment model
        treatment_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=seed_value)
        if dml_type == 'orthogonal':
            treatment_model = Ridge()

        if (cv_sampling == 'stratified') & (use_kbins):
            y_binned = pd.qcut(y_te, q=cv_nbins, labels=False)
            skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=seed_value)
            folds = [(train_idx, test_idx) for train_idx, test_idx in skf.split(X_te, y_binned)]
        else:
            folds = cv_folds

        # initialize DML for continuous treatment and outcome
        if dml_type == 'linear':
            dml = LinearDML(model_y=outcome_model, model_t=treatment_model, cv=folds)
        elif dml_type == 'forest':
            dml = CausalForestDML(model_y=outcome_model, model_t=treatment_model, cv=folds)
        elif dml_type == 'orthogonal':
            dml = DMLOrthoForest(model_Y=outcome_model, model_T=treatment_model, global_res_cv=folds)
        
        treatment_effect, lower_bound, upper_bound = {}, {}, {}
        for feature in tqdm(X_te_tf.columns):
            T = X_te_tf.copy()[feature]
            # discretize treatment in N bins
            if use_kbins:
                kbins = KBinsDiscretizer(n_bins=nbins, encode='ordinal', strategy='uniform')
                T = kbins.fit_transform(T.values.reshape(-1, 1)).flatten()
            X_covariates = X_te_tf.drop(columns=[feature])
            # fit DML model with the outcome (Y), treatment (T), and covariates (X)
            dml.fit(Y=y_te, T=T, X=X_covariates)
            treatment_effect[feature] = dml.effect(X_covariates)
            lower_bound[feature] = dml.effect_interval(X_covariates, alpha=0.05)[0]
            upper_bound[feature] = dml.effect_interval(X_covariates, alpha=0.05)[1]
        treatment_effect = pd.DataFrame(treatment_effect, index=X_te.index)
        upper_bound = pd.DataFrame(upper_bound, index=X_te.index)
        lower_bound = pd.DataFrame(lower_bound, index=X_te.index)

        ite_values_cp[loc_name][freq]['treatment_effect'] = treatment_effect
        ite_values_cp[loc_name][freq]['upper_bound'] = upper_bound
        ite_values_cp[loc_name][freq]['lower_bound'] = lower_bound
        ite_values_cp[loc_name][freq]['X'] = X_te
        ite_values_cp[loc_name][freq]['y'] = y_te
        ite_values_cp[loc_name][freq]['y_pred'] = pd.Series(outcome_model.predict(X_te_tf), index=X_te.index)

        ite_values_cp[loc_name][freq]['Time'] = X_te['Time'].values
        ite_values_cp[loc_name][freq]['Location'] = X_te['Location'].values
        ite_values_cp[loc_name][freq]['Frequency'] = X_te['Frequency'].values

use_target_transform = use_target_transform_
columns_not_in_model = columns_not_in_model_

### Plot treatment effects over time for conformal intervals

In [ ]:
def plot_ite_over_time(ite_data, freq, loc_name, win_size=6, max_features=10, cmap='tab10', skip_neg=False, color_other='lightgray', sort_statistic='max', show_features=True, effect_type='raw'):
    """
    Function to plot SHAP values over time with the top N SHAP features stacked
    and actual vs predicted values in a separate axis.
    
    Parameters:
    - ite_values_cp: A dictionary containing SHAP values and other related data
    - freq: The frequency to plot (index into the frequency dimension)
    - loc_name: The location to plot (key to select the location from ite_values_cp)
    - win_size: The size of the rolling window to smooth the SHAP values (default: 6)
    - max_features: The maximum number of top SHAP features to display (default: 10)
    - cmap: The colormap to use for feature plots (default: 'tab10')
    - color_other: Color for the 'Other' category (default: light gray)
    - sort_type: Type of sorting to use for the top features (default: 'max')
    - show_features: Whether to show the features in a separate axis (default: True)
    """
    
    # Get DataFrame for treatment effects
    ite_to_plot = ite_data[loc_name][freq].copy()['treatment_effect']
    ite_to_plot = ite_to_plot.assign(
        **{
            'Time': ite_data[loc_name][freq]['Time'],
        }
    )

    if effect_type == 'proportional':
        threshold = 1e-3
        for feature in ite_to_plot.drop(columns='Time', errors='ignore').columns:
            ite_to_plot[feature] = np.where(ite_data[loc_name][freq]['X'][feature] > threshold, ite_to_plot[feature] / ite_data[loc_name][freq]['X'][feature], 0)
    elif effect_type == 'signed':
        for feature in ite_to_plot.drop(columns='Time', errors='ignore').columns:
            ite_to_plot[feature] = ite_to_plot[feature] * np.sign(ite_data[loc_name][freq]['X'][feature])

    # Step 1: Calculate mean absolute treatment effect for each feature (sorted in descending order)
    if sort_statistic == 'median':
        sorted_ite_values = ite_to_plot.drop(columns=['Time']).abs().agg(np.median).sort_values(ascending=False)
    else:
        sorted_ite_values = ite_to_plot.drop(columns=['Time']).abs().agg(sort_statistic).sort_values(ascending=False)

    # Step 2: Select top N features based on mean SHAP values
    top_features = sorted_ite_values.index[:max_features].tolist()
    other_features = sorted_ite_values.index[max_features:].tolist()
    if len(other_features) == 1:
        top_features.extend(other_features)
        other_features = []

    # Sort top features in ascending order by mean treatment effect
    sorted_top_features = sorted_ite_values[top_features].sort_values().index

    # Step 4: Create a new dataframe with only top N features and 'Other'
    ite_top_N = ite_to_plot[['Time'] + list(sorted_top_features)]  # Sorted top features

    # Step 5: Apply rolling window smoothing **before** splitting into positive and negative
    for feature in ite_top_N.drop(columns='Time', errors='ignore').columns:  # Skip 'Time' column
        ite_top_N.loc[:, feature] = ite_top_N[feature].rolling(window=win_size, center=True).mean()
    
    # Step 6: Split into positive and negative SHAP values **after rolling**
    ite_pos = ite_top_N.copy()
    ite_neg = ite_top_N.copy()

    # Replace negative SHAP values with 0 in the positive dataframe, and vice versa
    for feature in ite_top_N.drop(columns='Time', errors='ignore').columns:  # Skip 'Time' column
        ite_pos[feature] = ite_top_N[feature].apply(lambda x: x if x > 0 else 0)
        ite_neg[feature] = ite_top_N[feature].apply(lambda x: x if x < 0 else 0)

    # Step 7: Define the colormap
    color_map = plt.get_cmap(cmap)
    num_features = len(ite_top_N.drop(columns='Time', errors='ignore').columns)  # Number of features to plot
    colors = [color_map(i) for i in range(num_features)][::-1]

    # Step 8: Plot SHAP stacked area chart (Second Axis)
    fig, ax = plt.subplots(nrows=3 if show_features else 2, ncols=1, sharex=True, figsize=(16,16) if show_features else (16, 12), dpi=150)
    if show_features:
        fig.subplots_adjust(hspace=0.4)
    else:
        fig.subplots_adjust(hspace=0.2)

    # Initialize baselines for stacking (positive and negative)
    pos_baseline = np.zeros(len(ite_pos['Time']))
    neg_baseline = np.zeros(len(ite_neg['Time']))
    # Add each feature to the plot
    for i, feature in enumerate(ite_top_N.drop(columns='Time', errors='ignore').columns):  # Skip 'Time' column
        # Stack positive SHAP values (upwards)
        ax[0].fill_between(ite_pos['Time'], 
                           pos_baseline,  # Previous cumulative baseline
                           pos_baseline + ite_pos[feature],  # New cumulative value
                           label=feature, color=colors[i],
                           linewidth=0.05)
        # Update positive baseline
        pos_baseline += ite_pos[feature]
        
        # Stack negative SHAP values (downwards)
        if not skip_neg:
            ax[0].fill_between(ite_neg['Time'], 
                               neg_baseline,  # Previous cumulative baseline
                               neg_baseline + ite_neg[feature],  # New cumulative value
                               label=None, color=colors[i],
                               linewidth=0.05)
            # Update negative baseline
            neg_baseline += ite_neg[feature]

    # Customize first axis
    ax[0].grid(axis='both', color=[.7, .7, .7], linestyle='-', linewidth=.5)
    ax[0].set_axisbelow(True)
    if effect_type == 'proportional':
        ylabel = 'Proportional Treatment Effects of Features\non Prediction Intervals'
    elif effect_type == 'signed':
        ylabel = 'Signed Treatment Effects of Features\non Prediction Intervals'
    else:
        ylabel = 'Raw Treatment Effects of Features \non Prediction Intervals'
    if win_size > 1:
        ylabel = ylabel + f" (MA, window={win_size}hr)"
    ax[0].set_ylabel(ylabel, fontsize=12)

    ax[0].set_title(f"Location: '{loc_name}', Frequency: {freq}", fontsize=22)
    ax[0].tick_params(axis='x', labeltop=False, labelbottom=True)
    ax[0].annotate(
        f"Note: Treatment effects aren't necessarily additive. The effect of changing multiple features\nsimultaneously might not be the sum of their individual effects, especially if features interact", 
        xy=(0.01, 0.02), xycoords='axes fraction', fontsize=10, fontstyle='oblique'
    )

    # Flip entire legend
    handles, labels = ax[0].get_legend_handles_labels()
    ax[0].legend(handles[::-1], labels[::-1], loc='upper left', bbox_to_anchor=(1., 1.02), ncol=1, fontsize=9)

    # Step 9: Plot predicted vs actual values (First Axis)
    ax[1].plot(ite_data[loc_name][freq]['Time'], 
               ite_data[loc_name][freq]['y'].rolling(window=win_size, center=True).mean(), 
               lw=2.5, label=f"{cp_method.upper()} Interval Estimate", zorder=0)
    ax[1].plot(ite_data[loc_name][freq]['Time'], 
               ite_data[loc_name][freq]['y_pred'].rolling(window=win_size, center=True).mean(), 
               lw=1.5, label="Model-based Reconstruction", zorder=10)
    ax[1].grid(axis='both', color=[.7, .7, .7], linestyle='-', linewidth=.5)
    ax[1].set_axisbelow(True)
    ax[1].set_ylabel(f"Estimated Prediction Intervals using {cp_method.upper()} Method\nwith Step-wise Update of Residuals Distribution", fontsize=12)
    ax[1].legend(loc='upper left', bbox_to_anchor=(1., 1.02), ncol=1, fontsize=9)
    ax[1].tick_params(axis='x', labeltop=False, labelbottom=True)

    # Step 10: Show top N features
    if show_features:
        if len(other_features) > 0:
            colors = colors[1:]
        colors = colors[::-1]
        for i,feature in enumerate(top_features):
            ax[2].plot(ite_data[loc_name][freq]['X']['Time'], ite_data[loc_name][freq]['X'][feature] / ite_data[loc_name][freq]['X'][feature].abs().max(), lw=1.2, label=feature, color=colors[i])
        ax[2].grid(axis='both', color=[.7, .7, .7], linestyle='-', linewidth=.5)
        ax[2].set_axisbelow(True)
        ax[2].set_ylabel('Normalized Feature Values', fontsize=12)
        ax[2].legend(loc='upper left', bbox_to_anchor=(1., 1.02), ncol=1, fontsize=9)

    plt.show()

In [ ]:
loc_idx = 2
freq_idx = 16
win_size = 48
max_features = 99

freq = X['Frequency'].unique()[freq_idx]
loc_name = locations_for_conformal_prefit[loc_idx]
plot_ite_over_time(
    ite_values_cp, 
    freq=freq, 
    loc_name=loc_name, 
    cmap='tab20', 
    win_size=win_size, 
    max_features=max_features, 
    sort_statistic='median',
    show_features=False,
    effect_type='raw',
)